In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "yeoSXsVuzsTf"
      },
      "source": [
        "# Week 5 - Deep learning with Transformers"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Vxq_lY1Wz2AK"
      },
      "source": [
        "Some preliminary set-up."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 401,
          "referenced_widgets": [
            "72691438bd3443e496380c56af496eaa",
            "44408e7021a34ed7b5aeeed803f48f30",
            "e54ba4ac5c6949d7bcb97eb9729d6ada",
            "335a51f7dd5548c3947a6288495c1737",
            "2bd6ec97d073442eb08d141c07fd199d",
            "7ffad741bcf84ea59f3a41239cd92045",
            "38e443a3d6124361b5fc1acaaeaeb93e",
            "3b037f3257684895ba25778397aeee19",
            "2fa2d24e07cf4c4cba4cd296ba9281b3",
            "c278bbf65b6d43c0b82af9a3cab5ef9e",
            "3f549e3d727043358247df4eddcc7c07"
          ]
        },
        "id": "rK63fnkquGHY",
        "outputId": "1921863d-7063-46ab-ca22-dc0c11f592ea"
      },
      "outputs": [],
      "source": [
        "! pip install ipywidgets rich seaborn torch datasets transformers tokenizers sentencepiece sacremoses --quiet\n",
        "\n",
        "%matplotlib inline\n",
        "\n",
        "import os\n",
        "os.environ[\"TOKENIZERS_PARALLELISM\"] = \"false\"\n",
        "\n",
        "import torch\n",
        "from torch import nn\n",
        "import math\n",
        "from functools import partial\n",
        "from pathlib import Path\n",
        "from tqdm import tqdm\n",
        "import rich\n",
        "from typing import List, Tuple, Optional, Dict, Any\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "import numpy as np\n",
        "import transformers\n",
        "import tokenizers\n",
        "import datasets\n",
        "import zipfile\n",
        "from huggingface_hub import hf_hub_download\n",
        "\n",
        "sns.set()\n",
        "\n",
        "# define the device to use\n",
        "DEVICE = torch.device(\"cuda\") if torch.cuda.is_available() else torch.device(\"cpu\")\n",
        "rich.print(f\"Device: [red]{DEVICE}\")\n",
        "\n",
        "# control verbosity\n",
        "transformers.logging.set_verbosity_error()\n",
        "datasets.logging.set_verbosity_error()\n",
        "\n",
        "# define support functions\n",
        "def load_glove_vectors(filename = \"glove.6B.300d.txt\") -> Tuple[List[str], torch.Tensor]:\n",
        "    \"\"\"Load the GloVe vectors. See: `https://github.com/stanfordnlp/GloVe`\"\"\"\n",
        "    path = Path(hf_hub_download(repo_id=\"stanfordnlp/glove\", filename=\"glove.6B.zip\"))\n",
        "    target_file = path.parent / filename\n",
        "    if not target_file.exists():\n",
        "        with zipfile.ZipFile(path, 'r') as zip_ref:\n",
        "            zip_ref.extractall(path.parent)\n",
        "\n",
        "        if not target_file.exists():\n",
        "            print(f\"Available files:\")\n",
        "            for p in path.parent.iterdir():\n",
        "                print(p)\n",
        "            raise ValueError(f\"Target file `{target_file.name}` can't be found. Check if `{filename}` was properly downloaded.\")\n",
        "\n",
        "    # parse the vocabulary and the vectors\n",
        "    vocabulary = []\n",
        "    vectors = []\n",
        "    with open(target_file, \"r\") as f:\n",
        "        for l in tqdm(f.readlines(), desc=f\"Parsing {target_file.name}...\" ):\n",
        "            word, *vector = l.split()\n",
        "            vocabulary.append(word)\n",
        "            vectors.append(torch.tensor([float(v) for v in vector]))\n",
        "    vectors = torch.stack(vectors)\n",
        "    return vocabulary, vectors"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 105,
          "referenced_widgets": [
            "864803498d9d485b854f6dc22bf8e7f7",
            "e276dfc72b4f4b42a5a93241c7dcd0b4",
            "30caa70b8e2b4ca28af34fb14cd0cf86",
            "233c55aac32242f6be5085631989f7c8",
            "c1d0a6be0e0b402696f8f759ecf692e2",
            "09b9bc4a5757400fb352de17b2eb1cab",
            "9ef088f4f6d244738be3a42da4396635",
            "eb07e619043940ecb3dd33ec3f38b03c",
            "d7522bc3874c4285b33ca65febd3f2d5",
            "586aa53e10754cb8948e3636da7aef9c",
            "e0ea063acfe245c783f720279a8004cd"
          ]
        },
        "id": "gtB1kzl_uGHm",
        "outputId": "107a3b51-d3f5-4e5f-8894-626e95df4bc8"
      },
      "outputs": [],
      "source": [
        "# prepare data for the later cells\n",
        "glove_vocabulary, glove_vectors = load_glove_vectors()\n",
        "rich.print(f\"glove_vocabulary: type={type(glove_vocabulary)}, length={len(glove_vocabulary)}\")\n",
        "rich.print(f\"glove_vectors: type={type(glove_vectors)}, shape={glove_vectors.shape}, dtype={glove_vectors.dtype}\")\n",
        "\n",
        "# add special tokens\n",
        "special_tokens = ['<|start|>', '<|unknown|>', '<|pad|>']\n",
        "glove_vocabulary = special_tokens + glove_vocabulary\n",
        "glove_vectors = torch.cat([torch.randn_like(glove_vectors[:len(special_tokens)]), glove_vectors])\n",
        "\n",
        "# tokenizer for GloVe\n",
        "glove_tokenizer = tokenizers.Tokenizer(tokenizers.models.WordLevel(vocab={v:i for i,v in enumerate(glove_vocabulary)}, unk_token=\"<|unknown|>\"))\n",
        "glove_tokenizer.normalizer = tokenizers.normalizers.BertNormalizer(strip_accents=False)\n",
        "glove_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "eKoXNCEfuGHr"
      },
      "source": [
        "# Language Modelling and Transformers\n",
        "\n",
        "___\n",
        "## Content\n",
        "\n",
        "* I. Text to vectors\n",
        "* II. Language models\n",
        "* III. Attention mechanism\n",
        "* IV. Transformers\n",
        "* V. Applications of Transformer-based language models\n",
        "\n",
        "\n",
        "___\n",
        "## Introduction\n",
        "\n",
        "Since its introduction ([\"Attention is All You Need\", Wasrani et al. (2016)](https://arxiv.org/abs/1706.03762)), Transformers have overtaken the field of Machine Learning. Initially applied to translation tasks, Transformers pre-trained on vast amounts of unlabelled data such as BERT and GPT have been acquired as central components in most of the modern natural language processing (NLP) systems. Transformers power question answering (QA) models, machine translation services, search engines and chat bots. Independently of the language applications, the Transformer is also a versatile neural architecture and, therefore, has found success outside the field of NLP. Transformers are rapidly being adopted in image processing ([\"An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale\", Dosovitskiy et al. (2021)](https://arxiv.org/abs/2010.11929)), in reinforcement learning ([\"A Generalist Agent\", Reed et al. (2022)](https://arxiv.org/abs/2205.06175)), video generation ([\"VideoGPT: Video Generation using VQ-VAE and Transformers\", Yan et al. (2021)](https://arxiv.org/abs/2104.10157)), and more. In the following sections, we will first introduce the basics of NLP (tokenization, token embeddings, language modelling), introduce the attention mechanism. In the second part, we will study the Transformer architecture and apply it to NLP tasks.\n",
        "\n",
        "___\n",
        "## I. Text to vectors\n",
        "\n",
        "In the previous labs, we have applied deep learning to processing images encoded as RGB pixels. We found that processing arrays of RGB pixels using convolutional neural network was effective. In NLP, other neural interfaces are required to enable plugging text into neural networks. Raw text cannot trivially be plugged-in neural networks. In this section we show how to convert text units or *tokens* into vectors and introduce the notion of text vector spaces.\n",
        "\n",
        "### I.a. Tokenization\n",
        "\n",
        "In [alphabetic languages](https://en.wikipedia.org/wiki/List_of_writing_systems), text can be decomposed into various types of units or *tokens*: characters, syllables, words or even sentences. Each tokenization system comes with vocabulary $\\mathcal{V}$ that references all known symbols. \n",
        "\n",
        "The choice of tokenizer is a tradeoff between the size of the vocabulary and the number of tokens required to encode a sentence. For instance, character-level tokenizers result in a smaller vocabulary size (only 128 character when using ASCII encoding) than other tokenizers. Word-based tokenizers encode text using fewer tokens than the other tokenizers but require a much larger vocabulary, which still might miss words seen at test time. Sub-words tokenizers such as [WordPiece](https://arxiv.org/abs/2012.15524) and [byte-pair encoding (BPE)](https://arxiv.org/abs/1508.07909) are a tradeoff between character-level and word-level encoding. They have progressively taken over the field as they provide two main advantages: (i) good tradeoff between vocabulary size and encoding length, (ii) open-ended vocabulary. \n",
        "\n",
        "Below we tokenize one sentence using word-level, character-level and sub-word-level tokenizers. In each case, the output corresponds to a sequence of indexes corresponding to the position of the given token in the vocabulary."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 837,
          "referenced_widgets": [
            "f01a9c20f21547b6926e9eb44b8f43af",
            "91759f59f8604393a0d72c623b1b97af",
            "06a4618bda4949f2ace26450b1d8052e",
            "55783e273f894123b18c1ab84fe996a1",
            "8f96e5e63c61463181ad8daed8f104e0",
            "8befeadf18114bdba529e65426de91e2",
            "8ee9eb7d0c8c4f2998dc41835994e442",
            "89d19eb4b1a243bf8318b037f3eb1eb2",
            "4a41a1e705b549f8b094e5a1432ea389",
            "f1e394f5978f44de9d7f539e78699582",
            "d1f5bcde8d754af18fdd052c98a6d81b",
            "9b39591d155d4d5b9bf7b9c37c928332",
            "52a04a50c3bb491dabd849005adfe60b",
            "1d9d2cf9cd59425497adf32e4dd59261",
            "882871406b0043229ea41e2bc7e20cd9",
            "2af41c4c23e24e3db73a9003a32d0097",
            "ddd780b848994a3eaea4257b5b3f6dee",
            "85fa69e6cb8c4b84b8bcb7c36bf2d8dd",
            "9b601f47611b4281b2b2d1fcab39c38b",
            "670ce1a5a4a3477e970db3210192ec4a",
            "28ac8463e3f14e899b7e16553d991d41",
            "71a9ceb690724a8ab688acbf67c68113",
            "d667b4d49dfe49bebbf39dba10ada96d",
            "ed1c5ae4218e48bc924fc615e416721f",
            "0869911afc8549d9bd4ce0e7f97bdc0c",
            "242c86c1e834490897262daa0ef5dfcd",
            "389af45e82694958b597e0c25529f6c0",
            "a303620e1ed142658667b32cee4b7e70",
            "0330f9b958f34c46a34e62b47568fa77",
            "4bde8be94bb945b7869e4fa261d686dd",
            "9b6c07e98b4c4e95bdddc7dac23d69cf",
            "971ee1ec3e25467fa861cf30207f25a7",
            "15b3668ae54c4fb88b9d67d5756122c4",
            "5fd3ce3c4f1e4751b6e1567b1d0cc447",
            "c5bcb4f5e0b740f6adaaecbc9b38a8e0",
            "36a31d0fa707478285f094582fb2b090",
            "8b0b3d0a39df422ab85e98147252c37d",
            "20d9a77f209d473bb1d2e751d706913e",
            "8a2dcb7fe90040328cacd70146c48202",
            "b4ef3d467ecb4d63a3a8e90b9db13fb3",
            "b2966a02f9b74b7895c525b6fb21597d",
            "67eb4a8763f648c1a4934e076c64bb2b",
            "81bd92c195424af2b552038bb2240473",
            "fee18278d97f4191a086db8dc185409b",
            "b29bd3a6f57d414ea8c2625317daf851",
            "2da10880d1da4b7eba255333d171d653",
            "74d518f82ecb4021ade1a713de64104b",
            "8b54d237a7cc43beaeb6a1edee13714e",
            "148fd6dd13ca40199c14bc396a741d83",
            "5cb53426cc29476dadb4c2fb88c5b5ad",
            "3ce1683b100b4e249a4272220b0b6bed",
            "583320fb40344b9bbb1887848173ca45",
            "9b421d1eb87a4d9fbde251053fcf6c2c",
            "bcd5a9c91e52470c9c285fda8a659a94",
            "fa07548194e244b8b655381e073abb9d",
            "9cc17f6d1a40454b8db2052160168a12",
            "ec288e953b244616befab8429457c77b",
            "c6b3661a3447492ab9e5b8d1b3aae05e",
            "3586240e502f4255ab0d3d66e79b449e",
            "64f9c055090a4861b33f8ed15a858daf",
            "2d966f4595464c49be7f5b323cf54b1c",
            "17d2e815f3ca46fa82167ccd1eb85b2a",
            "f9ac22e242984baeaae8ddae3748003c",
            "2de4643e661d448caaa7a1a7e660c6b0",
            "588dfe8a747a4cd2a9ca35bba8898808",
            "6d3fa316dfa24d20bda288ddfb71b4c8",
            "31949d7fd38f4c20b8bb734dbf9aaa3b",
            "636cfa48a2c647efa9ada29b42c37a35",
            "bc150147b2c845f3aead1c4b1fa09c61",
            "a0f74a508afc44ed9c618e1d181807a5",
            "689de7097be44956ac7d9a5736b8e5bb",
            "57c21a5483ec40a998a7ef66301a41ae",
            "4dce807720b442788b75eecf25174e86",
            "978de6521f3d4b8b9ac305b9db7a0ff1",
            "bfa2bb61bc534200bdb61bc28bc09b53",
            "2cb7d45a1a804c109e67e9a6c50976ee",
            "2ba33d1e19ac48149fc2c78e1baba2a9",
            "31e124e44206472ab49233418434b9ce",
            "8d664b21ce974899bb6a7aaaea21a69e",
            "e2036efdcaab46719ab2b57baf01e2a2",
            "1a511d9f81cd434ea9a4a0baa1ad43f2",
            "acfe14fcd0fc484b957fe97cfcb34c4c",
            "a8650804aeaf4a37b9732b890791d06d",
            "d3f574de1ff448178358f3a4ddf803c9",
            "94d4edee3f7a406281f694241eb143ab",
            "84ccee50a10d47f392d094a53c5223d9",
            "5b85631ac5a446e5884020fb7ba4b96f",
            "fc44ba4fc74f472a95cbeab1ea179b89",
            "847410262bdd45919fdb4dbb18a5df2c",
            "3828d19ebc43415f8ff87568418ec9b2",
            "f79a56e0fe024b9682ec79d134201412",
            "a5e970c0cf8a48ae8d2cb45058887d6b",
            "aab826eacf3b429a8b67bb4c42698154",
            "3369c01acc964d9b8b5738e59b05b6b4",
            "15280581483b47f18aa57296f25d8334",
            "bc51d76c3a854586b7b4832771a112bf",
            "4429b3f77f2941468d72ba1b17821d9a",
            "d8afe82dd0454aeeb34099fd21ccf726",
            "04a785e5695545f8905306feee92b618",
            "59add760ed794047bbb548f5c09e64ac",
            "ff32978b71364a37a6724a5ee508bed3",
            "43f85ac6c8ae4044a0e4443d7da79eb3",
            "1e9772f564fa4d4390518664a1595197",
            "aedd4dc023644d5cb76f33462ae965c7",
            "6835c929cd844a7f882765f68bd8a7e2",
            "1de84e67a3b34cddb1c73beae7888833",
            "f3e74d9727564ed3ac609ebb3ec74737",
            "16ebe063385841faba7f79ae57dad474",
            "f34c4b62a07949b68758d6f6ffd1b31f",
            "7ce42569074943d3af0ca12b2ac351d7",
            "c11372c27bb34a03a26971a77791141a",
            "65fd5c5e9b2c4cbdb4fe0c3314ae29bb",
            "cefc16b8f9c94fc0aee7ba72a1c4aeb0",
            "d35f83b7433e4f1bb857789c9738bf21",
            "999d7a37dd8843b1aa9920c0da638c81",
            "5c1987d5e4824dda8818f2d2187a0666",
            "340c96ed160c42b191291545dfe6848c",
            "8f1bf0c58d1e493a8a00e96ce09590da",
            "a45abec670d242799e43d5badb75fa99",
            "91e3ec446c5d4030853dca426e304f9c",
            "718e86634d1343559659a545f36abb42"
          ]
        },
        "id": "MPEQDuq4uGHz",
        "outputId": "4683777f-3247-482d-c852-f7903f307b1b"
      },
      "outputs": [],
      "source": [
        "# Example sentence with rare English words and non-english words\n",
        "sentence = \"It is jubilating to see how élégant my horse has became\"\n",
        "rich.print(f\"Input sentence: [bold blue]`{sentence}`\")\n",
        "\n",
        "# Define multiple tokenizers\n",
        "tokenizer_ids = {\n",
        "    \"Word-level\": glove_tokenizer,\n",
        "    \"WordPiece\": \"bert-base-cased\",\n",
        "    \"BPE\": \"distilgpt2\",\n",
        "    \"Character-level\":  \"google/byt5-small\",\n",
        "    }\n",
        "\n",
        "# iterate through the tokenizers and decode the input sentences\n",
        "for tokenizer_name, tokenizer in tokenizer_ids.items():\n",
        "    # intialize the tokenizer (either)\n",
        "    if isinstance(tokenizer, str):\n",
        "        # init a `transformers.PreTrainedTokenizerFast`\n",
        "        tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer)\n",
        "        vocab_size = tokenizer.vocab_size\n",
        "    else:\n",
        "        # use the provided `tokenizers.Tokenizer``\n",
        "        vocab_size = tokenizer.get_vocab_size()\n",
        "\n",
        "    # Tokenize\n",
        "    token_ids = tokenizer.encode(sentence, add_special_tokens=False)\n",
        "    if isinstance(token_ids, tokenizers.Encoding):\n",
        "        token_ids = token_ids.ids\n",
        "\n",
        "    # Report\n",
        "    rich.print(f\"[red]{tokenizer_name}[/red]: sentence converted into {len(token_ids)} tokens (vocabulary: {vocab_size} tokens)\")\n",
        "    rich.print(f\"Tokens:\\n{[tokenizer.decode([t]) for t in token_ids]}\")\n",
        "    rich.print(f\"Token ids:\\n{[t for t in token_ids]}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "DNH3dQ3VuGH1"
      },
      "source": [
        "### I.b Embeddings\n",
        "\n",
        "A tokenizer transforms fragments of text into list of integers that maps a vocabulary. We assign one vector of dimension $d$ to each item in the vocabulary of size $N_\\mathcal{V}$, this results in a matrix $E$ of dimension ${N_\\mathcal{V} \\times d}$. Converting a fragment of text into a sequence of vector representations can be done by tokenizing the text, and then looking up the embedding vector for each token, which is equivalent to *one-hot encoding* the tokens and performing a matrix multiplication using $E$. Given $\\mathbf{t}_1, \\ldots, \\mathbf{t}_L$ the sequence of one-hot encoded tokens, this is equivalent to\n",
        "$$\n",
        "\\mathbf{w}_i = E  \\mathbf{t}_i ,\n",
        "$$\n",
        "In the code below, we encode the sentence `Hellow world!` using a BPE tokenizer and a set of embedding of dimension `hdim`."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 271,
          "referenced_widgets": [
            "fae2e3a4035d403dad3baef4802a9258",
            "bd052d320db8492c9b3aa1cf29e4a9cf",
            "55ed66dd456e4f19ae98ce739da552b8",
            "a4befd44d0b5402e8bcb61f64162ff43",
            "f08a4af32b364cbaa774b179c87f737a",
            "74acdd9bc09c438ab1f48c8c762c4a07",
            "f8eaa261c4df44cea66453f6233555f9",
            "c59172344f204280a756b106f69ed793",
            "e35b435f72d4403e9ac00852fad7eef7",
            "380c55c2680a4f85a81cfb266eace834",
            "94ac791468ea4909a84c353d5bc7611d",
            "713c08d937ca4e31a8b8f6942a487b7a",
            "a4424cb16ec849ee96df431f52a57e29",
            "5f1eef839eb24e25bbf667d0c23d2378",
            "bad437a15b834e3e8ca6ca67b4860b7d",
            "836bd677b7b040e2812fa3d3cf39b491",
            "3e842b6e81c041419f630f7bbf69e0d2",
            "b81956f3a32f4d8cb6dbe07a62867627",
            "7ff7ecf2e5f44961becb04faa0a3048b",
            "cad503bc0c3346e2abf5b25d355305d7",
            "4772a52c663241ba9b5314d887fd47de",
            "1c28b4c4a1bd44ddbed1028e3a755bb5",
            "cdb2e3a16db04678a4a6d5d23931dacc",
            "6c2c388a465f447a976d3427b59bad7a",
            "b173b9397ed748b98aa18e9b4b739fca",
            "4ffa8b39a34a4433bcce70f713bd7f2a",
            "62f64323983d426c8762984fd25bb02e",
            "09e77e9a34f24fedbdb90b2112f27347",
            "5cef548a19104d2a910071036dccc588",
            "b611fcfb99e84693954b41743fbf321a",
            "8d2ad2b28f1644f09ff21d95df9bf0c0",
            "3a7a4945d0f04ce5ac54621bf1d02698",
            "af69668241644ef4b5cb06223a565d9f",
            "fc631832218144c4b29f776ebba56193",
            "0f938c22289d45d9bdd7929ed1a97626",
            "b9474845c2d24c6f91420bb23ca0e149",
            "2066c6d9fe2a4d6f876e81fbbf64e653",
            "868d87d61432416fb23969b8405cc628",
            "67e91a4c9376417dbb9916e83f5ccdd4",
            "9ca45bbcbdbe4eab98cb67ed1249d8fb",
            "88b39f446a4d46f5a5748d8bdf4311cf",
            "7fdf1d094b4e4693acc53477fd2379c2",
            "00046c6611c940058bac1aa75da16f95",
            "08995f7f72f14dd8a22565ee4ae41215"
          ]
        },
        "id": "MKD5CuNLuGH2",
        "outputId": "f0a4835d-f378-4048-a7c0-409198a91649"
      },
      "outputs": [],
      "source": [
        "hdim = 5 # embedding dimension\n",
        "tokenizer = transformers.AutoTokenizer.from_pretrained(\"bert-base-uncased\") # tokenizer\n",
        "sentence = \"Hello World!\" # input text\n",
        "embeddings = torch.randn((tokenizer.vocab_size, hdim)) # embedding matrix\n",
        "rich.print(f\"Embeddings (shape): {embeddings.shape}\")\n",
        "token_ids = tokenizer.encode(sentence, add_special_tokens=False, return_tensors=\"pt\")[0]\n",
        "rich.print(f\"Tokens ids (shape): {token_ids.shape}\")\n",
        "vectors =  torch.nn.functional.one_hot(token_ids, tokenizer.vocab_size).float() @ embeddings # equivalent to a `nn.Linear` layer\n",
        "rich.print(f\"Vectors (shape): {vectors.shape}\")\n",
        "rich.print(f\"List of tokens and their corresponding vectors:\")\n",
        "for t,v in zip(token_ids, vectors):\n",
        "    token_info = f\"[blue]{tokenizer.decode(t):5}[/blue] (token id: {t:4})\"\n",
        "    rich.print(f\" * {token_info} -> {v}\")\n",
        "\n",
        "# NB: in practice, we use the simpler interface `torch.nn.Embedding``\n",
        "# embeddings = torch.nn.Embedding(tokenizer.vocab_size, hdim)\n",
        "# vectors = embeddings(token_ids)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "BZ9GcEgOuGH4"
      },
      "source": [
        "### I.c Word vectors\n",
        "\n",
        "<img src=\"images/word2vec-intuition.png\" alt=\"Word2vec: translations in the vector spaces correspond to linguistic concepts (gender, verb tense, association between concepts)\" width=\"800px\"/>\n",
        "\n",
        "\n",
        "Word2vec ([\"Efficient Estimation of Word Representations in Vector Space\", Mikolov et al. (2013)](https://arxiv.org/abs/1301.3781)) converts words into vector representations, which are learned using the Skip-Gram algorithm. Intuitively, The algorithm is based on the idea that words that appear together are related to each other.\n",
        "\n",
        "The word vector space allows to use the inner product to compare words, and arithmetic operations to manipulate word representations. For instance, in a well-defined word vector space, the concept \"king\" can be translated into \"queen\" by applying a linear transformation and the vector `vec(\"captial\") - vec(\"country\")` was found to correspond to the relative concept `\"capital city of a country\"` (see above illustration (*Image credits: https://www.tensorflow.org/tutorials/word2vec*)).\n",
        "\n",
        "\n",
        "**Experiment** In the first cells, we have downloaded the [GloVe word vectors](ttps://github.com/stanfordnlp/GloVe) from [\"GloVe: Global Vectors for Word Representation\", Jeffrey Pennington et al. (2014)](https://arxiv.org/abs/1902.11004). GloVe are trained using a Skip-Gram objective on a large collection of documents. "
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 431
        },
        "id": "NQHCpsoXuGH7",
        "outputId": "6a26788a-0ab5-431f-bfb7-454b626d67d4"
      },
      "outputs": [],
      "source": [
        "def word2vec(\n",
        "        word: str,\n",
        "        vocabulary:List[str],\n",
        "        vectors: torch.Tensor\n",
        "    ) -> Optional[torch.Tensor]:\n",
        "    \"\"\"Convert a word into a vector\"\"\"\n",
        "    word = word.lower()\n",
        "    if word in vocabulary:\n",
        "        word_idx = vocabulary.index(word)\n",
        "        return vectors[word_idx]\n",
        "    else:\n",
        "        return None\n",
        "\n",
        "def vec2words(\n",
        "        vec: torch.Tensor,\n",
        "        k=5,\n",
        "        *,\n",
        "        vocabulary:List[str],\n",
        "        vectors: torch.Tensor,\n",
        "        exclude_vecs: List[torch.Tensor] = None,\n",
        "    ) -> Tuple[List[str], torch.Tensor]:\n",
        "    \"\"\"Retrieve the nearest word neighbours for an input vector\"\"\"\n",
        "\n",
        "    # compute the similarity between `vec`and all the vectors in `glove_vectors`\n",
        "    similarity = vectors @ vec\n",
        "\n",
        "    # potentially filter out some vocabulary entries\n",
        "    if exclude_vecs is not None and len(exclude_vecs):\n",
        "        mask = None\n",
        "        for e in exclude_vecs:\n",
        "            mask_ = (vectors == e[None, :]).all(dim=1)\n",
        "            if mask is None:\n",
        "                mask = mask_\n",
        "            else:\n",
        "                mask |= mask_\n",
        "        similarity.masked_fill_(mask=mask, value=-math.inf)\n",
        "\n",
        "    # return the ids of the nearesrt neighbours given the similarity\n",
        "    nearest_neighbour_ids = torch.argsort(-similarity)[:k]\n",
        "\n",
        "    # retrieve the corresponding words in the `vocabulary``\n",
        "    return [vocabulary[idx] for idx in nearest_neighbour_ids], similarity[nearest_neighbour_ids]\n",
        "\n",
        "# register the vocab and vectors args\n",
        "glove_args = {'vocabulary':glove_vocabulary, 'vectors':glove_vectors}\n",
        "\n",
        "# Nearest neighbours\n",
        "rich.print(\"[red]Nearest neighbour search:\")\n",
        "for word in [\"king\", \"queen\", \"dog\", \"France\"]:\n",
        "    rich.print(f'Nearest neighbours of the word \"{word}\":')\n",
        "    word_vec = word2vec(word, **glove_args)\n",
        "    words, similarities = vec2words(word_vec, k=5, **glove_args, exclude_vecs=[word_vec])\n",
        "    rich.print(f\"Words: {words}\")\n",
        "    rich.print(f\"Similarities: {similarities}\")\n",
        "\n",
        "# Word analogies\n",
        "rich.print(\"\\n[red]Vector arithmetic:\")\n",
        "cases = [\n",
        "    [(\"+\", \"king\"), (\"-\", \"man\"), (\"+\", \"woman\")],\n",
        "    [(\"+\", \"denmark\"), (\"-\", \"france\"), (\"+\", \"paris\")],\n",
        "    [(\"+\", \"pakistan\"), (\"-\", \"belgium\"), (\"+\", \"brussels\")],\n",
        "]\n",
        "for operations in cases:\n",
        "    # current location in the vector space\n",
        "    location = 0\n",
        "    rich.print(f\"Vector Translation: [blue]0 {' '.join(f'{d} {v}' for d,v in operations)} = \")\n",
        "    for sign, word in operations:\n",
        "        # retrieve the `vec(word)``\n",
        "        vec = word2vec(word, **glove_args)\n",
        "        if vec is None:\n",
        "            raise ValueError(f\"Unknown word `{word}`\")\n",
        "\n",
        "        # parse the direction (+/-)\n",
        "        direction = {\"+\": 1, \"-\": -1}[sign]\n",
        "\n",
        "        # apply the vector transform to the current location\n",
        "        location  +=  direction * vec\n",
        "\n",
        "    # return the nearest neighbours of the end location\n",
        "    exclude_list = [word2vec(w, **glove_args) for _, w in operations]\n",
        "    words, similarities = vec2words(location, k=5, exclude_vecs=exclude_list, **glove_args)\n",
        "    rich.print(f\"Words: {words}\")\n",
        "    rich.print(f\"Similarities: {similarities}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "0ZnUwuv0uGIA"
      },
      "source": [
        "**Beyond word2vec**  The Skip-Gram model allows to learn meaningful word prepresentations and arithmetic in the resulting vector space allow manipulating concepts. Ultimately, we are interested in learning representations that represent larger text fragments such as sentences, paragraphs or documents. Doing so require combining multiple vectors, which can be done by exploiting arithmetic in the vector space, or by combining word-vectors using deep neural networks, such as Transformers!\n",
        "\n",
        "___\n",
        "## II. Language models\n",
        "\n",
        "We have seen how to encode text into sequences of tokens, seen how to convert tokens into vectors using a set of embeddings and experimented with a GloVe word vector space. In this section we will see how to model text at the sentence, pragraph or even document level using language models.\n",
        "\n",
        "### II.a Language Modelling\n",
        "\n",
        "*Figure: Left-to-right language models*\n",
        "![Autoregressive left-to-right language model](images/ar-lm.png)\n",
        "\n",
        "**Autoregressive factorization** Language models aim at grasping the underlying linguistic structure of a fragment of text: whereas word vectors model words independently of each others, a language model tracks the grammatical and semantic relationships between word tokens. Given a piece of text encoded into tokens $\\mathbf{w}_{1:T} = [\\mathbf{w_1}, \\ldots, \\mathbf{w}_T]$ a *left-to-right* language model describes $\\mathbf{w}_{1:T}$ with the following factorization:\n",
        "$$\n",
        " p_\\theta(\\mathbf{w}_{1:T}) = \\prod_{t=1}^T p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t}) \\ ,\n",
        "$$\n",
        "where $\\theta$ is a model parameter. The above *autoregressive* factorization describes a *recursive* function $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$, which is shared across all the time steps. In the above figure, we represent a left-to-right language model with dependencies represented by arrows for fixed steps $t=3$ and $t=4$. Because of this choice of factorization, a language model defines a graphical model where each step $t$ depends on all the previous steps $<t$ and the conditional $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ models the dependendies between the context $\\mathbf{w}_{<t}$ and the variable $\\mathbf{w}_t$.\n",
        "\n",
        "**Other factorizations** Autoregressive models are not required to adopt a left-to-right factorization and other forms of factorizations can be implemented (right-to-left or arbitrary permutations). See [\"XLNet: Generalized Autoregressive Pretraining for Language Understanding\", Yang et al. (2019)](https://arxiv.org/abs/1906.08237) for an example."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "W5dhjnjNuGID"
      },
      "source": [
        "*Figure: Categorical distribution over the possible next tokens given the context*\n",
        "![Categorical distribution](images/categorical-dist.png)\n",
        "\n",
        "**Distribution of the possible next tokens** The distribution $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ is a categorical distribution defined on the whole token vocabulary $\\mathcal{v}$. Without loss of generality, we denote $f_\\theta(\\mathbf{w}_t, \\mathbf{w}_{<t})$ the function with parameter $\\theta$ that parametrize:\n",
        "$$\n",
        "p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t}) = \\frac{\\exp f_\\theta(\\mathbf{w}_t, \\mathbf{w}_{<t})}{\\sum_{\\mathbf{v} \\in \\mathcal{V}} \\exp f_\\theta(\\mathbf{v}, \\mathbf{w}_{<t})} \\ .\n",
        "$$\n",
        "The distribution $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ assigns a probability for each token in the vocabulary to appear right after the context $\\mathbf{w}_{<t}$. As pictured in the above figure, many completion are possibles under the model $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$.\n",
        "\n",
        "**Language models learn contextual representations** Assigning a probability to each word in the vocabulary requires learning abstract representations of the context $\\mathbf{w}_{<t}$. For instance, in the horse example, predicting the word \"fast\" will be easier to predict if some knowledge of the grammatical rules and common sense is acquired. In this example example, the model needs to learn that $\\mathbf{w}_4$ must be an adjective, and that this adjective can be attributed to a horse. Therefore, the function $f_\\theta$ must acquire a non-trivial representation of the context to make sensible token predictions$."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "UjOlZ-wSuGIE"
      },
      "source": [
        "**Sampling** At each step $t$, the $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ implements a categorical distribution defined on the vocabulary $\\mathcal{V}$. Sampling or *generating* text can by iteratively sampling tokens, as showed in the pseudo-code bellow:\n",
        "```python\n",
        "ws = [] # placeholder for all the samples w_t\n",
        "for t in range(T):\n",
        "    wt_logits = f(ws, theta) # logits of p(w_t | w_{<t})\n",
        "    wt = Categorical(logits=wt_logits).sample() # sampled w_t\n",
        "    ws.append(wt) # append the new sample to be used as input in the next step\n",
        "```\n",
        "Sampling is often expensive, as it requires one evaluation of the function $f_\\theta$ for each step and cannot be trivially parallelized.\n",
        "\n",
        "**Training** As long as the transition function $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ is differentiable (i.e., using neural networks), a language model can be  trained via maximum likelihood, e.g. maximizing the log-likelihood with the loss:\n",
        "$$\n",
        "L = - \\log p_\\theta(\\mathbf{w}_{1:T}) = - \\sum_{t=1}^T \\log p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})\n",
        "$$\n",
        "Each term $p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ can be evaluated using the observed variables $\\mathbf{w}_t$ and $\\mathbf{w}_{<t}$ (no sampling is required) and thus training of auto-regressive models is fast when the evaluation of $f_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$ can be parallelized."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "nH7KUquguGIE"
      },
      "source": [
        "*Figure: Bidirectional language models*\n",
        "![Masked language model](images/masked-lm.png)\n",
        "\n",
        "**Bidirectional and masked language models** Autoregressive language models learn to predict a token $\\mathbf{w}_t$ given the context up to the step $t-1$. One can also use a [pseudo likelihood](https://en.wikipedia.org/wiki/Pseudolikelihood), where $\\mathbf{w}_t$ is not only conditioned on the preceeding tokens $\\mathbf{w}_{<t}$, but also on the next tokens $\\mathbf{w}_{>t}$. This defines a bidirectional language model, which factorizes as\n",
        "$$\n",
        "L_\\theta(\\mathbf{w}_{1:T}) = \\prod_{t=1}^T p_\\theta(\\mathbf{w}_t \\mid \\mathbf{w}_{-t}) \\ ,\n",
        "$$\n",
        "where $\\mathbf{w}_{-t}$ represent the set of tokens $\\mathbf{w}_{1:T} \\backslash \\{ \\mathbf{w}_t \\}$. We call it pseudo because this likelihood is not forming a valid distribution (because the graph formed by $\\mathbf{w}_{1:T}$ is not a directed acyclic graph (a DAG)).  Bidirectional language models such as [ELMo (\"Deep contextualized word representations\", Peters et al. (2018))](https://arxiv.org/abs/1802.05365), learn token representation contextualized on the whole context.\n",
        "\n",
        "In the case, of bidirectional language models, the context $\\mathbf{w}_{-t}$ corresponds to the whole sequence of tokens with the predicted element masked out. It is possible to generalize the bidirectional factorization to masking out one or more tokens. In that case, we consider a model $p_\\theta(\\mathbf{w}_m \\mid \\mathbf{w}_{-m})$ where $m$ is a set of indices of the tokens being predicted and $-m$ is the set of the other tokens. This is notably the approach adopted in [\"BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding\", Delvin et al. (2018)](https://arxiv.org/abs/1810.04805)."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "NQT5jAR5uGIF"
      },
      "source": [
        "### II.b Recurrent Neural Networks\n",
        "\n",
        "*Figure: Left-to-right recurrent neural network. We highlight the information flowing from the context \"My horse is\" to the predicted word \"very\".*\n",
        "![Recurrent Neural Network](images/recurrent-lm-activated.png)\n",
        "\n",
        "**Recurrent neural networks (RNNs)** implement a recursive function $f_\\theta$ using neural networks, which makes them a particularly good fit for sequential data. In the general setting, RNNs model the acquired knowledge at time $t$ using an additional variable $\\mathbf{h}_t$ of dimension $d_h$ (*hidden state*). The hidden state at step $t-1$ is updated with the information extracted from the observation $\\mathbf{w}_t$ using a function\n",
        "$$\n",
        "h_\\theta: (\\mathbf{w}_{t}, \\mathbf{h}_{t-1}) \\rightarrow \\mathbf{h}_{t} \\ ,\n",
        "$$\n",
        "which can be imlemented using an arbitrary neural network that takes the tuple $(\\mathbf{w}_{t}, \\mathbf{h}_t)$ as input and returns a new hidden state $\\mathbf{h}_{t+1}$. RRNs can be applied to parametrize language models by projecting the hidden state $\\mathbf{t}$ into the vocabulary space using a projection matrix $\\mathbf{F} \\in \\mathcal{R}^{V \\times d_h}$. This results in parameterizing the transition distribution as\n",
        "$$\n",
        "p_\\theta(\\cdot \\mid \\mathbf{w}_{<t}) = \\mathrm{Softmax}( \\mathbf{h}_t \\mathbf{F}^T)\n",
        "$$\n",
        "In the above figure, we showcase how a standard RNN can be applied to implement a left-to-right language model, and annotated the diagramm with the function $h_\\theta(\\mathbf{w}_{t}, \\mathbf{h}_{t-1)})$ and the projection matrix $\\mathbf{F}$.\n",
        "\n",
        "**Long Short-Term Memory (LSTM) networks** A standard RNN suffers from [the vanishing gradients problem](http://neuralnetworksanddeeplearning.com/chap5.html#the_vanishing_gradient_problem) which gives challenges in saving memory over longer sequences. To combat these issues the gated hidden units were created. The two most prominent gated hidden units are the [Long Short-Term Memory (LSTM, Hochreiter and Schmidhuber. (1997))](https://dl.acm.org/doi/10.1162/neco.1997.9.8.1735) cell and the Gated Recurrent Unit (GRU), both of which have shown increased performance in saving and reusing memory in later timesteps. RNNs coupled with gated mechanisms are less prone to the problem of vanishing gradients, and can therefore model dependencies over longer number of steps."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "9qGgbRBcuGIF"
      },
      "source": [
        "*Figure: bi-directional recurrent neural network. We highlight the information flowing from the context \"My horse is\" to the predicted word \"very\" (left-to-right), and the information flowing from the context \"fast\" (right-to-left).*\n",
        "![Recurrent Neural Network](images/bidirectional-lm-activated.png)\n",
        "\n",
        "**Bi-directional recurrent neural networks** Using two RNNs running in reverse direction allows building bidirectional language models. The distribution $p_\\theta(\\mathbf{x}_t \\mid \\mathbf{x}_{-t})$ can be parameterized as\n",
        "$$\n",
        "p_\\theta( \\cdot \\mid \\mathbf{x}_{-t}) = \\mathrm{Softmax}((\\mathbf{h}^\\mathrm{forward}_t + \\mathbf{h}^\\mathrm{reverse}_t) F^T) \\ ,\n",
        "$$\n",
        "where the hidden state  $\\mathbf{h}^\\mathrm{bi}_t = \\mathbf{h}^\\mathrm{forward}_t + \\mathbf{h}^\\mathrm{reverse}_t$ defines hidden state contextualized on all the tokens but $\\mathbf{w}_t$. \n",
        "\n",
        "This is the strategy adopted by [ELMo (\"Deep contextualized word representations\", Peters et al. (2018))](https://arxiv.org/abs/1802.05365), which popularized learning deep contextualized representations as a pre-training step, and at the samd time, started a [tradition of naming models after Seame Street characters](https://www.theverge.com/2019/12/11/20993407/ai-language-models-muppets-sesame-street-muppetware-elmo-bert-ernie)."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ZbzQlrGXuGIF"
      },
      "source": [
        "**Experiment: train your own LSTM language model**\n",
        "\n",
        "> **NB**  *training on CPU is very slow. If you don't have access to a GPU, it will be difficult to train a model that generate acceptable samples. In the end of the notebook, we will use pre-trained models directly, so feel free to skip this experiment. **You still nee to implement the loss in the training loop***"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 623,
          "referenced_widgets": [
            "b4ca9d8896d6480093c780f9c0f4d3ee",
            "ed3abc2e209f4590a4626efa3d2d24be",
            "693c8df85dde460b87f0624d33e196ed",
            "ef5ff713eb5c41738bdd73921418308c",
            "b2be2c8915c842a3a197c00b5ceeb77f",
            "9780793b16aa4029ae550481db873a7d",
            "eae0b201409d4f60bb60cbdb7ad9a217",
            "3e24127c11c243e9aa7c300d4950f2d1",
            "6f98a870b3684a79977334c15c9d1324",
            "bee129673ec34431932b80148b50aab0",
            "9ba8816c2f51491a8596bb33fc8998d8",
            "4c2f7522315c4606b6e14ea7a7d1a607",
            "091851ddeeb6437aa1a8cc8fbe2bc17c",
            "8b0669653acf44bc877c194a5e57944a",
            "33d93bd8482841f5ad2bc99317c85c27",
            "74e6b1af5e5942578decd8a2d806f5e7",
            "ddb0ac228105475a83b334351aed3439",
            "a642a8054e0a4ba08a516b18498ee769",
            "35d7392e1f7441269689a5260050bf8d",
            "21e3b941f0b24664895ad9913206d11c",
            "7b964fbfd635438f96ff1858ca33c0e5",
            "49d1ba4b752145f4ac0668dfdb4b40b0",
            "2f2fad7073e347e28b71cc61ecb53513",
            "da110a0f535848838ac3018c9bd5603c",
            "ad3776cc4d204d2daaa0bc18c24bdcba",
            "38efabdd23f74921951c71ae079405d7",
            "9ca3d1acbf904aa2b7f9f1f1475a5e9f",
            "5199e127b316467d872958d3d03fb8c5",
            "1ba2f6eb67f342c4aca41c7b8624ddde",
            "88f0a8f7296e47af8f751190bc11b065",
            "ec57f9449406445a9e91e35c0236da0b",
            "80d43815d1fb49afba8f36b0cd86a762",
            "c26834a8a1474c839203335c9c57237d",
            "b62ce40a257541c496cb5c8c914f670e",
            "df3806b3fc674e4f9f899d5dbb6a4115",
            "e98979e4e72441019858ec61946f934f",
            "7da43125ff734d4580f56def5c2a24a6",
            "92df20c724504084af9595139f251978",
            "3ef01336fc77403789e6c7bf2fe7fbd4",
            "6709c3d560714b06b77382f279440ef6",
            "674a1753257541edb0be46d7811c6e72",
            "4826faba9b6049929733e16542881e14",
            "973c58059452472697894eab30adea3b",
            "cdc952769a3841de91d4d49513b5f4c8",
            "1732733c760947fb920e9de7606705ef",
            "7e7492861d2d4334aaa522829fe52d0a",
            "5a37cba6a0da4612bbcc177d0d547d31",
            "1838ebcc7ea34641938901ab9415cf58",
            "00f958e144f346cc8ff25825d293f1d3",
            "135d819a8d7f4e3ba85bed56b318a0d3",
            "601c8305985b422b84ef3ad5cf7561e9",
            "671bd251ea704b32894d76b15c824772",
            "9a019409c8154d869f71a5a8ebcc4ddf",
            "255c7719289d40ee8d7858e7c97f62b8",
            "d4447eac662145188017fe281566cfea",
            "b8d4aa7e1c6a4e4898f3fba303e318bd",
            "b701e33b12a14bac98bab4d888e72c43",
            "43f0edc85a3e447f92b22820c71a7b01",
            "e231320115e544239d9a800d9cd32094",
            "5ba11688b1a34d1298a3c0b164b7b3cd",
            "5d32f0e96b11429fbd8f2d7aaa3cabe8",
            "737eb4f444f5460eadbedf755e488804",
            "93f474326a6e41809bdf32621655423e",
            "edd8accd2cd444ac80e7d7dd9e93c605",
            "e0c61f515ead4c4b9025c7c085456131",
            "80f274ee73da4a2dbfb3d56d48636ef5",
            "63a8257eab9d4099a9611b13cdf8b20e",
            "739a02fcd76f4b7ab68f4044d38fbad5",
            "f101c0bc1fc34bbdbe6b386d7858b593",
            "fb68c9499ecb4403939127cd750718eb",
            "5661f01f86c849568a78e21c7fce7f5b",
            "5bcd549955304dbeba4003e13518ab74",
            "9a83be7115ca4b6dbb90715c551da421",
            "d78a21a570954c9589f7780f53000985",
            "7703ade855614a7099230146866d17e6",
            "75efbc49c6e84d9f8146c04de0f4daa8",
            "4f35888d5ef74153b2c1bf739cce1f9c",
            "e921dd034a844b888523be3de972db48",
            "d2daf033cc624958b2769a4349a04a01",
            "5ca4915e26b34b1d8d125f63a9b3f8ff",
            "8975b0dad9b34d4bb3d654e833af9915",
            "62930768884441de957525daceb027de",
            "8111135a8f6f4c6b8e634fe978045a56",
            "1b32d1e2681d4561840a29c55d11bc9f",
            "85ae16107d9443c2bbced1a4df0ca71b",
            "fda6478d993342a798d281d6da9d8cd0",
            "f832dc2bf7e048ccb168cc96bc8fb8f2",
            "a94420be4e1d4aa49774c2fce210aeaa",
            "1a1d215fecb049429b7cac703cbb010d",
            "238bca108c114c5bb1bb20c6a9126bee",
            "5135f2da12774e49b3941a75ee539fb0",
            "d1e2d720723b4d99bda65860a15ad427",
            "298186b7e41e4c3c9047ed35d8ca5b79",
            "ca79683bd37243f98b5f10f2e6f373c4",
            "c923a57cac3a44389ad8b8bf0920a8aa",
            "9f63d34305c544cda90ae4a9ee60449b",
            "f4318ba53a2b49ce95af89537538d815",
            "39722132ac22429fb0d99e5545566715",
            "f7163be6070842c0abd1cc8c5af5682c",
            "e0da976d36eb42d18df29ca6ff382e34",
            "f4742bc69760471090d5b0c0015e6a78",
            "62ca2598d0ff440483940669dde3f193",
            "2d5966ea44ec4e469a758037f90aaa3b",
            "b4d53b1db23e46278f2a68258594c9da",
            "52ce268c3a4d4c1da647dda194d2e511",
            "a1cb51aa9a3246eda0676252be68909c",
            "fd921171897a4ffc99562d675733b308",
            "3dee85fadf5648c6b8b109ee428783fa",
            "527265f507a946a283f7437e679fa909",
            "a10e589c393744f58fc09cc1b97c8df5",
            "90beda2982944894a9450e5a509daf2e",
            "60e62383e2ec45deaf196e18b02f39af",
            "fd2c6d3ac0754db3acad1e88ade2c86d",
            "c5f0c1a82e1f422983431c3463ef9604",
            "66b4340421ba4bb88797ed6ebc04c177",
            "981a2d85af5849b4802f2c32489c9100",
            "30df52fae0f74e1caa14dab8f4689aff",
            "89c56c8131c74168ad32930ab11500f1",
            "96906f2b2dcf4a34bcb6418d44751575",
            "bafda22907ec4b6e929dc553f936e18d",
            "785bfc66d62b4e109573f605b5fec286"
          ]
        },
        "id": "mzeY47_JuGIG",
        "outputId": "84ba5026-8361-45e5-b248-65733f95e42e"
      },
      "outputs": [],
      "source": [
        "max_dataset_size = 1000 # let's use a small subset for now,\n",
        "max_seq_size = 10 # and very short sequences\n",
        "\n",
        "# load and tokenizer the dataset\n",
        "def batch_tokenize(batch: List[Dict[str, Any]], max_length=max_seq_size, tokenizer: tokenizers.Tokenizer = None, key:str=\"text\") -> torch.Tensor:\n",
        "    texts = batch[key]\n",
        "    encodings = tokenizer.encode_batch(texts)\n",
        "    return {\"token_ids\": [x.ids[:max_length] for x in encodings]}\n",
        "\n",
        "# load AG News, take a subset of `max_dataset_size` rows and tokenize\n",
        "dataset = datasets.load_dataset(\"ag_news\")\n",
        "dataset = datasets.DatasetDict({split: dset.select(range(max_dataset_size)) if len(dset) > max_dataset_size else dset for split, dset in dataset.items()})\n",
        "dataset = dataset.map(partial(batch_tokenize, tokenizer=glove_tokenizer), batched=True, num_proc=2, batch_size=10)\n",
        "rich.print(dataset)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "_x4tdP88uGIH"
      },
      "outputs": [],
      "source": [
        "class RNNLM(torch.nn.Module):\n",
        "    \"\"\"A simple implementation of a language model using RNNs.\"\"\"\n",
        "    def __init__(self, vectors:torch.Tensor):\n",
        "        super().__init__()\n",
        "        # register the embeddings\n",
        "        self.embeddings = torch.nn.Embedding(*glove_vectors.shape)\n",
        "        self.embeddings.weight.data = glove_vectors\n",
        "\n",
        "        # register the LSTM\n",
        "        self.rnn = torch.nn.LSTM(\n",
        "            input_size=glove_vectors.shape[1],\n",
        "            hidden_size=glove_vectors.shape[1],\n",
        "            num_layers=1,\n",
        "            batch_first=True,\n",
        "        )\n",
        "\n",
        "        # project the output of the LSTM (hidden state) back to the vocabulary space\n",
        "        self.proj = nn.Linear(glove_vectors.shape[1], glove_vectors.shape[0], bias=False)\n",
        "        # init the projection using the embeddings weights\n",
        "        self.proj.weight.data = glove_vectors\n",
        "\n",
        "    def forward(self, token_ids: torch.Tensor, retain_ws:bool=False) -> torch.Tensor:\n",
        "        # convert the tokens into vectors\n",
        "        ws = self.embeddings(token_ids)\n",
        "\n",
        "        # store the word vectors for debugging\n",
        "        if retain_ws:\n",
        "          ws.retain_grad()\n",
        "          self.ws = ws\n",
        "\n",
        "        # shift the input `ws` right\n",
        "        w0 = torch.zeros((ws.shape[0], 1, self.embeddings.weight.shape[1]),\n",
        "                         device=self.embeddings.weight.device, dtype=torch.long)\n",
        "        ws_shifted = torch.cat([w0, ws[:, :-1]], dim=1)\n",
        "\n",
        "        # call the RNN: w_{-1:T-1} -> h{1:T}\n",
        "        hidden_states, _ = self.rnn(ws_shifted)\n",
        "\n",
        "        # project the hidden state to the vocabulary space\n",
        "        logits = self.proj(hidden_states)\n",
        "        return logits\n",
        "\n",
        "    def sample(\n",
        "            self,\n",
        "            batch_size:int=1,\n",
        "            num_steps:int=10,\n",
        "            temperature: float=1.0,\n",
        "            prevent_repetitions: bool=False\n",
        "        ):\n",
        "        token_ids = torch.empty((batch_size, 0), device=self.embeddings.weight.device, dtype=torch.long)\n",
        "        for t in tqdm(range(num_steps), desc=f\"Sampling {num_steps} steps..\"):\n",
        "            logits = self.forward(token_ids)\n",
        "            logits_t = logits[:, -1:] / temperature\n",
        "            if prevent_repetitions and t > 0:\n",
        "                # mask the last generated tokens to avoid repetitions\n",
        "                logits_t.scatter_(index=token_ids[:,-1:, None], dim=2, value=-math.inf)\n",
        "            p_wt = torch.distributions.Categorical(logits=logits_t)\n",
        "            tokens_t = p_wt.sample()\n",
        "            token_ids = torch.cat([token_ids, tokens_t], dim=1)\n",
        "        return token_ids\n",
        "\n",
        "\n",
        "# init RNN initialized from GloVe vectors\n",
        "# delete the checkpoint if you get `PytorchStreamReader` error\n",
        "checkpoint_file = Path(\"rrn-lm.ckpt\")\n",
        "rnn = RNNLM(glove_vectors)\n",
        "if checkpoint_file.exists():\n",
        "    # checkpoint_file.unlink() # delete the checkpoint by un-commenting this line\n",
        "    rnn.load_state_dict(torch.load(checkpoint_file, map_location=\"cpu\"))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "2h9uwSmzuOdZ"
      },
      "source": [
        "**Testing** Let's make sure the autoregressive constrains are enforced ($\\mathbf{h}_t$ only depends on $\\mathbf{w}_{<t}$). We take differentiate a loss that depends only on a step $t = t'$ for each element $t'$ of the input batch and visualize the gradients with regards to the input word vectors (right after the embedding layers) $\\mathbf{w}_{1:T}$. The gradient map tells use which input positions are influencing the differentiated output position."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 303
        },
        "id": "l_EeZagOuOde",
        "outputId": "cda1be39-d96a-4715-d838-33506f3740af"
      },
      "outputs": [],
      "source": [
        "# Test whether the autoregressive constrains (h_t only depends on w_{<t}) is enforced\n",
        "rnn.zero_grad()\n",
        "# get dummy token ids\n",
        "token_ids = torch.arange(0, 10)\n",
        "token_ids = token_ids[None].repeat(10, 1)\n",
        "# run through the RNN\n",
        "logits = rnn(token_ids, retain_ws=True)\n",
        "\n",
        "# compute a loss for a which depends only on step `t=i`\n",
        "loss_locations = torch.arange(0, 10)[:, None, None].expand(10, 1, logits.shape[-1])\n",
        "loss = logits.gather(index=loss_locations, dim=1).mean()\n",
        "\n",
        "# backward pass and retrieve the gradients with respect to the word vectors w_{1:T}\n",
        "loss.backward()\n",
        "grad_magnitude = rnn.ws.grad.norm(dim=2)\n",
        "rnn.ws = None\n",
        "\n",
        "# visualize the gradient\n",
        "grad_magnitude[grad_magnitude==0] = -math.inf # the the gradient that are exactly zero to -inf for the sake of visualization\n",
        "grad_magnitude = grad_magnitude.detach().cpu().numpy()\n",
        "plt.imshow(grad_magnitude, sns.color_palette(\"viridis\", as_cmap=True))\n",
        "plt.colorbar()\n",
        "plt.grid(False)\n",
        "plt.xlabel(\"$t$ (input)\")\n",
        "plt.ylabel(\"$t'$ (loss)\")\n",
        "plt.title(\"Magnitude of the gradient w.r.t. $\\mathbf{w}_{1:T}$\")\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "yKm2KVC1E0Zh"
      },
      "source": [
        "**Exercise 1**: Explain in your own words what the plot shows. How would it look if we had a leakage of information from the future to the present?\n",
        "\n",
        "> *Insert your answer here.*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "kL-deg8-uOdf"
      },
      "source": [
        "**Exercise 2**: Implement the loss of the RNN language model.\n",
        "\n",
        "> answer in the code below."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "NK_EllpwuGIH",
        "outputId": "931540f5-9ec7-40f2-8650-48646b0421d0"
      },
      "outputs": [],
      "source": [
        "# use Adam optimizer\n",
        "optimiser = torch.optim.AdamW(rnn.parameters(), lr=1e-3)\n",
        "\n",
        "# define a data loader to iterate the dataset\n",
        "train_loader = torch.utils.data.DataLoader(\n",
        "    dataset['train'],\n",
        "    batch_size=64,\n",
        "    shuffle=True,\n",
        "    num_workers=2,\n",
        "    collate_fn=list,\n",
        ")\n",
        "\n",
        "# convert token_ids to tensors\n",
        "def make_batch(batch: List[Dict[str, Any]], max_length=24, key:str=\"token_ids\") -> torch.Tensor:\n",
        "    \"\"\"Collate the `token_ids` into a single tensor, pad to minimum length.\"\"\"\n",
        "    token_ids = [b[key] for b in batch]\n",
        "    max_length = min(max_length, *(len(x) for x in token_ids))\n",
        "    token_ids = [torch.tensor(x) for x in token_ids]\n",
        "    return torch.stack([x[:max_length] for x in token_ids])\n",
        "\n",
        "# If you don't have enough GPU memory, decrease the batch size, potentially along with the learning rate.\n",
        "rnn = rnn.to(DEVICE)\n",
        "num_steps = 0 # 5_000\n",
        "step = 0\n",
        "epoch = 0\n",
        "with tqdm(total=num_steps) as pbar:\n",
        "    while step < num_steps:\n",
        "        for batch in train_loader:\n",
        "            # concatenate the `token_ids``\n",
        "            batch_token_ids = make_batch(batch)\n",
        "            batch_token_ids = batch_token_ids.to(DEVICE)\n",
        "\n",
        "            # forward through the model\n",
        "            optimiser.zero_grad()\n",
        "            batch_logits = rnn(batch_token_ids)\n",
        "\n",
        "            # compute the loss (negative log-likelihood)\n",
        "            p_ws = torch.distributions.Categorical(logits=batch_logits) \n",
        "\n",
        "            # Exercise: write the loss of the RNN language model\n",
        "            # hint: check the doc https://pytorch.org/docs/stable/distributions.html#categorical\n",
        "            # NB: even with the right loss, training is slow and the generated samples won't be very good.\n",
        "            loss = ... # <- YOUR CODE HERE \n",
        "\n",
        "            # backward and optimize\n",
        "            loss.backward()\n",
        "            optimiser.step()\n",
        "            step += 1\n",
        "            pbar.update(1)\n",
        "\n",
        "            # Report\n",
        "            if step % 5 ==0 :\n",
        "                loss = loss.detach().cpu()\n",
        "                pbar.set_description(f\"epoch={epoch}, step={step}, loss={loss:.1f}\")\n",
        "\n",
        "            # save checkpoint\n",
        "            if step % 50 ==0 :\n",
        "                torch.save(rnn.state_dict(), checkpoint_file)\n",
        "            if step >= num_steps:\n",
        "                break\n",
        "        epoch += 1"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 253
        },
        "id": "uftZSXKLuGII",
        "outputId": "58158f7f-8d98-4163-d40a-dc607039b0b6"
      },
      "outputs": [],
      "source": [
        "# sample the RNN language model\n",
        "with torch.no_grad():\n",
        "    sample = rnn.sample(num_steps=10, batch_size=10, temperature=0.5, prevent_repetitions=True)\n",
        "    rich.print(glove_tokenizer.decode_batch(sample.detach().cpu().numpy()))"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "3N1ntVHquGII"
      },
      "outputs": [],
      "source": [
        "_ = rnn.cpu()\n",
        "# free-up memory if needed: delete the RNN model\n",
        "# del rnn"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "iH3nKK-cuOdy"
      },
      "source": [
        "**Exercise 3**: What would happen if the inputs of the RNN were not shifted to the right (in sample in the RNNLM class)?  \n",
        "\n",
        "> *Insert your answer here.*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "8gSqXyp5uGIJ"
      },
      "source": [
        "___\n",
        "## III. Attention mechanism and Transformers\n",
        "\n",
        "\n",
        "The attention mechanism was first introduce in machine learning for machine translation tasks [(\"Neural Machine Translation by Jointly Learning to Align and Translate\", Bahdanau et al. (2014))](https://arxiv.org/abs/1409.0473). Translation is a sequence-to-sequence problem which goal is to generate a translation of a source text. The attention mechanism was introduced to allow *attenting* the whole source text at any of the generation steps. We implement attention with the softmax function because it is a differential version of a hard zero-one attention.\n",
        "\n",
        "In this section, we will introduce the *scaled dot-product* self-attention mechanism and the Transformer architecture [(\"Attention is All You Need\", Wasrani et al. (2016))](https://arxiv.org/abs/1706.03762).\n",
        "\n",
        "### III.a Attention mechanism\n",
        "\n",
        "Attention has become a very important concept in deep learning beginning with [\"Neural Machine Translation by Jointly Learning to Align and Translate\", Badanau et. al. (2015)](https://arxiv.org/abs/1409.0473). The idea key idea in that paper is that when you translate a sentence from for example German to English then it is a good is helpful for the model when it generates  \n",
        "\n",
        "We define three variables:\n",
        "1. The query $\\mathbf{Q} = [\\mathbf{q}_i, \\ldots \\mathbf{q}_{T_\\mathbf{Q}}] \\in \\mathcal{R}^{T_\\mathbf{Q} \\times h_i}$, a sequence of vectors of length $T_\\mathbf{Q}$ and vector dimension $h_i$.\n",
        "1. The keys $\\mathbf{K} = [\\mathbf{k}_1, \\ldots \\mathbf{k}_{T_{\\mathbf{K}\\mathbf{V}}}] \\in \\mathcal{R}^{T_{\\mathbf{K}\\mathbf{V}} \\times h_i}$, a sequence of vectors of length $T_{\\mathbf{K}\\mathbf{V}}$ and vector dimension $h_i$.\n",
        "1. The values $\\mathbf{V} = [\\mathbf{v}_1, \\ldots \\mathbf{k}_{T_{\\mathbf{K}\\mathbf{V}}}] \\in \\mathcal{R}^{T_{\\mathbf{K}\\mathbf{V}} \\times h_o}$, a sequence of vectors of length $T_{\\mathbf{K}\\mathbf{V}}$ and of another dimension $h_o$, although in general we choose $h_i = h_o$.\n",
        "\n",
        "For each query, the attention mechanism returns a convex combinations of the values $\\mathbf{V}$. The attention mechanism is defined as\n",
        "$$\n",
        "\\mathrm{Attention}(\\mathbf{Q}, \\mathbf{K}, \\mathbf{V}) = \\mathrm{Softmax}\\left( \\frac{\\mathbf{Q} \\mathbf{K}^T}{\\tau} \\right) \\mathbf{V} \\ ,\n",
        "$$\n",
        "where $\\tau$ is a scaling parameter, set to $\\sqrt{h_i}$ in ([\"Attention is All You Need\", Wasrani et al. (2016)](https://arxiv.org/abs/1706.03762)). $\\mathrm{Attention}(\\mathbf{Q}, \\mathbf{K}, \\mathbf{V})$ is a sequence of $T_\\mathbf{Q}$ vectors, each of dimension $h_o$.\n",
        "\n",
        "The above expresson is equivalent to applying attention to each query vector $\\mathbf{q}$ separately. The output for each vector $\\mathbf{q}$ depends on the vector of weights $\\mathbf{\\Lambda} = \\mathrm{Softmax}\\left( \\frac{\\mathbf{q} \\mathbf{K}^T}{\\tau} \\right)$ with values $[\\lambda_1, \\ldots \\lambda_{T_{\\mathbf{K}\\mathbf{V}}}]$. The vector of weights $\\Lambda$ is a function of the inner-product $\\mathbf{q} \\mathbf{K}^T$, which defines a similarity metric between the the vectors $\\mathbf{q}$ and each of the key vectors $[\\mathbf{k}_1, \\ldots \\mathbf{k}_{T_{\\mathbf{K}\\mathbf{V}}}]$. Furthermore, as the weights sum to one, the output of the attention function is a convex combinations of the values:\n",
        "$$\n",
        "\\mathrm{Attention}(\\mathbf{q}, \\mathbf{K}, \\mathbf{V}) = \\sum_{i=1}^{T_{\\mathbf{K}\\mathbf{V}}} \\mathbf{\\lambda}_i \\mathbf{v}_i \\ .\n",
        "$$"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "7x3Z7U5IuGIJ"
      },
      "source": [
        "**Experiment** We will use the GloVe word vectors to illustrate the attention mechanism. \n",
        "\n",
        "We define queries and keys using the GloVe word vectors correspond to country names:\n",
        "$$\n",
        "\\begin{align}\n",
        "\\mathbf{Q} =& [\\mathrm{vec(\"Italy\")}, \\mathrm{vec(\"Korea\")}, \\mathrm{vec(\"Nicaragua\")}, \\ldots] \\\\\n",
        "\\mathbf{K} =& [\\mathrm{vec(\"China\")}, \\mathrm{vec(\"Russia\")}, \\mathrm{vec(\"Turkey\")}, \\ldots] \\ .\n",
        "\\end{align}\n",
        "$$\n",
        "The inner-product between paris is of country vectors will have a large values when the vectors are similar, this might happend when two countries are geographically or culturally close to each other because of the properties of the GloVe vectors. The last component required to apply the attention mechanism is set of value vectors $\\mathbf{V}$.\n",
        "\n",
        "The choice of values depends on the end problem, for this exercise we choose stay in the same theme as for the word2vec experiments. We choose the value vectors to represent the relative concept \"*capital city of a country*\", which correspond to the vector $\\mathrm{vec(\"Capital\\, city\")} - \\mathrm{vec(\"Country\")}$ in the gloVe vector space. In practice, for each country we set:\n",
        "$$\n",
        "\\begin{align}\n",
        "\\mathbf{V} =& [\\mathrm{vec(\"Beijing\")} - \\mathbf{K}_1, \\mathrm{vec(\"Moscow\")} - \\mathbf{K}_2, \\mathrm{vec(\"Ankara\")} - \\mathbf{K}_3, \\ldots] \\\\\n",
        "\\end{align}\n",
        "$$\n",
        "\n",
        "*Figure: Word vectors vec(\"capital city\") - vec(\"country\") represented in a vector space. Vectors might point in different direction depending on their position in the vector space.*\n",
        "![Attention analogies](images/attention-analogies.png)\n",
        "\n",
        "We represent the query, key and value vectors in the GloVe vector space in the above figure. The intuition is that the value vector $\\mathrm{vec}(\\text{\"capital city of a country\"})$ is not always the same depending on the country. Therefore, in this experiment, we will attempt to extract value vectors that are contextualized on the query. To do so, we will add the output of the attention (a mixture of vectors $\\mathrm{vec}(\\text{\"capital city\"}) - \\mathrm{vec}(\\text{\"country\"})$) to the vector representation of the query (a country). This corresponds to:\n",
        "\n",
        "$$\n",
        "\\mathrm{vec(\"Capital\\,city\\,of\\,the\\,query\\,country\")} =  \\mathrm{vec(\"Query\\,country\")} + \\underbrace{\\mathrm{Attention}(\\mathrm{vec(\"Query\\,country\")}, \\mathbf{K}, \\mathbf{V})}_{\\sum_{i=1}^{T_{\\mathbf{K}\\mathbf{V}}} \\mathbf{\\lambda}_i \\mathbf{v}_i}\n",
        "$$\n",
        "\n",
        "First, let's investigate the attention weights $\\lambda_1, \\ldots, \\lambda_{T_{\\mathbf{K}\\mathbf{V}}}$ for each query $\\mathbf{q}_1, \\ldots, \\mathbf{q}_\\mathbf{Q} $ separately:"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "Ee0NwTe1uGIK",
        "outputId": "fd242ba4-4d73-4592-d053-f237af08163b"
      },
      "outputs": [],
      "source": [
        "# instantiate some embeddings\n",
        "embeddings = torch.nn.Embedding(*glove_vectors.shape)\n",
        "embeddings.weight.data = glove_vectors\n",
        "embeddings.weight.requires_grad = False\n",
        "\n",
        "def names_to_vectors(values: List[str]) -> torch.Tensor:\n",
        "    encodings = glove_tokenizer.encode_batch(values)\n",
        "    vectors = [embeddings(torch.tensor(e.ids)).mean(dim=0) for e in encodings]\n",
        "    return torch.stack(vectors)\n",
        "\n",
        "\n",
        "def plot_attention_weights(attention_weights, queries_labels, keys_labels, nrows = 2):\n",
        "    \"\"\"Plot the attention scores between each of the queries and the keys.\"\"\"\n",
        "    ncols = len(attention_weights) // nrows\n",
        "    fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize = (10,6), sharex=False, sharey=True, dpi=300)\n",
        "    colors = sns.color_palette(\"viridis\", as_cmap=True)\n",
        "    def normalize(x, values):\n",
        "        return (x -  values.min()) / (values.max() - values.min())\n",
        "    for k, ax in enumerate(axes.flat):\n",
        "        query_label = queries_labels[k]\n",
        "        attention_weights_k = attention_weights[k]\n",
        "        cols = [colors(normalize(x, attention_weights).detach().item()) for x in attention_weights_k]\n",
        "        ax.bar(keys_labels, attention_weights_k, color=cols)\n",
        "        plt.setp(ax.get_xticklabels(), rotation=45, ha=\"right\", rotation_mode=\"anchor\")\n",
        "        ax.set_title(f\"Query={query_label}\")\n",
        "        if k % ncols == 0 :\n",
        "            ax.set_ylabel(\"Attention score\")\n",
        "        if k >= ncols:\n",
        "            ax.set_xlabel(\"$\\mathbf{K}$\")\n",
        "    fig.tight_layout()\n",
        "    plt.show()\n",
        "\n",
        "# Define keys, queries and values\n",
        "queries_labels = [\"Italy\", \"Korea\", \"Nicaragua\", \"Canada\", \"Algeria\", \"India\"]\n",
        "keys_labels = [\"China\", \"Russia\", \"Turkey\", \"Japan\", \"Thailand\", \"Germany\", \"France\", \"Sweden\", \"Poland\", \"Nigeria\", \"Morocco\", \"Colombia\", \"Chile\", \"USA\", \"Pakistan\"]\n",
        "keys_cities_labels = [\"Beijing\", \"Moscow\", \"Ankara\", \"Tokyo\", \"Bangkok\", \"Berlin\", \"Paris\", \"Stockholm\", \"Warsaw\", \"Abuja\", \"Rabat\", \"Bogota\", \"Santiago\", \"Washington\", \"Islamabad\"]\n",
        "# convert to vectors\n",
        "Q = names_to_vectors(queries_labels)\n",
        "K = names_to_vectors(keys_labels)\n",
        "V = names_to_vectors(keys_cities_labels) - K\n",
        "\n",
        "\n",
        "# compute the attention weights for each query using a ´for` loop\n",
        "attention_weights = []\n",
        "for q in Q:\n",
        "    log_lambda_q = (q @ K.T) / math.sqrt(float(Q.shape[-1]))\n",
        "    attention_weights.append(log_lambda_q)\n",
        "attention_weights = torch.stack(attention_weights)\n",
        "\n",
        "# plot attention weights for each query\n",
        "plot_attention_weights(attention_weights, queries_labels, keys_labels)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "6g7Jqx4suGIK"
      },
      "source": [
        "**Exercise 4**:  In this example, what is the value of $T_{\\mathbf{K}\\mathbf{V}}$, $T_{\\mathbf{Q}}$, $h_i$, $h_o$ and $\\tau$ ?\n",
        "\n",
        "> * $T_{\\mathbf{K}\\mathbf{V}} = ...$ \n",
        "> * $T_{\\mathbf{Q}} = ...$ \n",
        "> * $h_i = ...$\n",
        "> * $h_o = ...$\n",
        "> * $\\tau = ...$"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "7jIxNVJouGIL"
      },
      "source": [
        "**Implementing the attention function** We obtained a set of attention weights for each query, concatenating them results in a 2D matrix that will display bellow. Let's implement the `attention` function in the cell bellow using the inputs vectors $\\mathbf{Q}, \\mathbf{K}, \\mathbf{V}$ and visualize the output vector. we use [`torch.einsum`](https://pytorch.org/docs/stable/generated/torch.einsum.html) to implement the sum $\\sum_{i=1}^{T_{\\mathbf{K}\\mathbf{V}}} \\mathbf{\\lambda}_i \\mathbf{v}_i$."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 760
        },
        "id": "0No1-SrduGIL",
        "outputId": "5b257c1a-2a3b-47a1-ce6e-af8d5c40336a"
      },
      "outputs": [],
      "source": [
        "\n",
        "def plot_attention_map(attention_map, queries_labels, keys_labels, print_values:bool=False, ax=None, color_bar:bool=True):\n",
        "    \"\"\"Plot the attention weights as a 2D heatmap\"\"\"\n",
        "    if ax is None:\n",
        "        fig, ax = plt.subplots(figsize = (10,6), dpi=300)\n",
        "    else:\n",
        "        fig = plt.gcf()\n",
        "    im = ax.imshow(attention_map, cmap=sns.color_palette(\"viridis\", as_cmap=True))\n",
        "    ax.grid(False)\n",
        "    ax.set_ylabel(\"$\\mathbf{Q}$\")\n",
        "    ax.set_xlabel(\"$\\mathbf{K}$\")\n",
        "    ax.set_yticks(np.arange(len(queries_labels)))\n",
        "    ax.set_yticklabels(queries_labels)\n",
        "    ax.set_xticks(np.arange(len(keys_labels)))\n",
        "    ax.set_xticklabels(keys_labels)\n",
        "    plt.setp(ax.get_xticklabels(), rotation=45, ha=\"right\",\n",
        "            rotation_mode=\"anchor\")\n",
        "\n",
        "    # Loop over data dimensions and create text annotations.\n",
        "    if print_values:\n",
        "        for i in range(len(queries_labels)):\n",
        "            for j in range(len(keys_labels)):\n",
        "                text = ax.text(j, i, f\"{attention_map[i, j]:.2f}\",\n",
        "                            ha=\"center\", va=\"center\", color=\"w\")\n",
        "\n",
        "    if color_bar:\n",
        "      fig.colorbar(im, fraction=0.02, pad=0.04)\n",
        "    fig.tight_layout()\n",
        "\n",
        "def attention(Q, K, V, tau=None):\n",
        "    \"\"\"A simple parallelized attention layer\"\"\"\n",
        "    if tau is None:\n",
        "        tau = math.sqrt(float(Q.shape[-1]))\n",
        "    assert Q.shape[-1] == K.shape[-1]\n",
        "    assert K.shape[0] == V.shape[0]\n",
        "    attention_map = Q @ K.T / tau\n",
        "    attention_weights = attention_map.softmax(dim=1)\n",
        "    return torch.einsum(\"qk, kh -> qh\", attention_weights, V), attention_weights\n",
        "\n",
        "# return the output of the attention\n",
        "output, attention_weights = attention(Q, K, V, tau=1.)\n",
        "\n",
        "# plot the attention weights\n",
        "plot_attention_map(attention_weights, queries_labels, keys_labels, print_values=True)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "jXI8BkIruGIL"
      },
      "source": [
        "**Exercise 5**:  What effect has the parameter $\\tau$ on the attention mechanism? What is happening when using a large value for $\\tau$? using a small value for $\\tau$? In the limits $\\tau \\rightarrow 0$ and $\\tau \\rightarrow \\infty$\n",
        "\n",
        "> *Insert your answer here*\n",
        "\n",
        "\n",
        "**Visualizing the output word vector** In the code below we use the code from the previous word2vec experiment to generate the nearest neighbour corresponding to the analogy: \n",
        "\n",
        "$$\\mathrm{vec(\"Capital\\,city\\,of\\,the\\,query\\,country\")} = \\mathrm{vec(\"Query\\,country\")} + \\mathrm{Attention}(\\mathrm{vec(\"Query\\,country\")}, \\mathbf{K}, \\mathbf{V})$$"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 287
        },
        "id": "9aKcB64-uGIM",
        "outputId": "9013631d-9b4b-42fa-bf9e-b4eb44a220ec"
      },
      "outputs": [],
      "source": [
        "# Report of the nearest neighbors of the vector `query + Attention(query, keys, values)``\n",
        "for i, attn_output in enumerate(output):\n",
        "    # z = query + Attention(qeury, keys, values)\n",
        "    z = Q[i] + attn_output\n",
        "    rich.print(f\"Nearest neighbors of [red]{queries_labels[i]}[/red] + [blue]Attention({queries_labels[i]}, K, V)\")\n",
        "    rich.print(vec2words(z, k=5, **glove_args, exclude_vecs=[word2vec(queries_labels[i], **glove_args)]))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "TnDCifTxuGIM"
      },
      "source": [
        "**Eercise 6** Would the results be different when shuffling the vectors $\\mathbf{K}$ and $\\mathbf{V}$ (with the same permutation for both vectors)?\n",
        "\n",
        "> *Insert your answer here*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "QLOpitlYuGIM"
      },
      "source": [
        "### III.b Self-attention\n",
        "\n",
        "*Figure: A self-attention layers allows attending all the sequence positions*\n",
        "![Self-attention allows attentind at all the sequence positions](images/self-attention.png)\n",
        "\n",
        "The attention mechanism is a transformation of a sequence of vectors $\\mathbf{Q}$ given all the values in the sequence $\\mathbf{V}$. In a self-attention layers, the attention layer is parameterized with transformations of the input sequence $\\mathbf{w}_{1:T}$ as parameters, which allows processing each vector in a sequence $\\mathbf{w}_{1:T}$ based on all the other locations. The output of the self-attention layer is a sequence of hidden states:\n",
        "$$\n",
        " \\mathbf{h}_{1:T} = \\mathrm{Attention} \\left(\\mathbf{Q}(\\mathbf{w}_{1:T}), \\mathbf{K}(\\mathbf{w}_{1:T}), \\mathbf{V}(\\mathbf{w}_{1:T}) \\right)\n",
        "$$\n",
        "\n",
        "**Illustration** Let's apply the self-attention to a list of word vectors, do you any structure emerging?"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "Gpvdqb05uGIN",
        "outputId": "97882708-b068-4316-e6df-c691d2e42ee3"
      },
      "outputs": [],
      "source": [
        "# Illustration of the above using code\n",
        "hdim = 300\n",
        "sentence = \"Copenhagen Denmark Stockholm Sweden Beijing China Tokyo Japan truck car bus\"\n",
        "token_ids = torch.tensor(glove_tokenizer.encode(sentence).ids)\n",
        "tokens = [glove_vocabulary[x] for x in token_ids]\n",
        "vectors = embeddings(token_ids)\n",
        "H, attention_map = attention(vectors, vectors, vectors)\n",
        "rich.print({\n",
        "    \"Q\": Q.shape,\n",
        "    \"K\": K.shape,\n",
        "    \"V\": V.shape,\n",
        "    \"H\": H.shape\n",
        "})\n",
        "\n",
        "# visualized the log of the attention map\n",
        "plot_attention_map(attention_map.log(), tokens, tokens)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "iEEYKKh8uOfC"
      },
      "source": [
        "**Exercise 7**: Comment on the structure of the attention map. Why is that the case?\n",
        "\n",
        "> *Insert your answer here*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "kdrvH3USuOfM"
      },
      "source": [
        "### IV.b Masked attention\n",
        "\n",
        "The self-attention layer allows computing the hidden state $\\mathbf{h}_{t}$ based on all the input vectors $\\mathbf{w}_{1:T}$. In language modelling, we want to enforce constrains on the dependencies of $\\mathbf{h}_t$ to allow left-to-right or masked factorizations. A attention mask $\\mathbf{M} \\in \\{0, -\\infty \\}^{T \\times T}$ is of the same dimension as the matrix $ \\mathbf{Q} \\mathbf{K}^T$ and can be utilized to enforce the attention weights $\\mathbf{\\lambda}_1, \\ldots, \\mathbf{{\\lambda}_T}$ to be zero wherever it is necessary. The masked attention mechanism is expressed as\n",
        "$$\n",
        "\\mathrm{Attention} \\left(\\mathbf{Q}, \\mathbf{K}, \\mathbf{V}, \\mathbf{M} \\right) = \\mathrm{Softmax}\\left( \\mathbf{M} + \\frac{\\mathbf{Q} \\mathbf{K}^T}{\\tau} \\right) \\mathbf{V} \\ .\n",
        "$$"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "l3uGReoIbqGb"
      },
      "source": [
        "**Exercise 8**:  Let's implement an attention mask corresponding to:\n",
        "1. Left-to-right language model $p(\\mathbf{w}_t \\mid \\mathbf{w}_{<t})$\n",
        "1. bidirection language model $p(\\mathbf{w}_t \\mid \\mathbf{w}_{-t})$\n",
        "1. right-to-left language model $p(\\mathbf{w}_t \\mid \\mathbf{w}_{t>})$\n",
        "\n",
        "> Answer in the code below.\n",
        "\n",
        "**NB** In the visualization bellow, we re-use the gradient map extracted from the left-to-right RNN language model.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 691
        },
        "id": "6z_qvaxjuGIN",
        "outputId": "aa01a8cd-ac82-4aef-e061-a9d4bc8b42f7"
      },
      "outputs": [],
      "source": [
        "def masked_attention(Q, K, V, tau=None, mask=None):\n",
        "    \"\"\"A simple masked attention layer\"\"\"\n",
        "    if tau is None:\n",
        "        tau = math.sqrt(float(Q.shape[-1]))\n",
        "    assert Q.shape[-1] == K.shape[-1]\n",
        "    assert K.shape[0] == V.shape[0]\n",
        "    attention_map = Q @ K.T / tau\n",
        "    if mask is not None:\n",
        "        attention_map = mask + attention_map\n",
        "    attention_weights = attention_map.softmax(dim=1)\n",
        "    return torch.einsum(\"qk, kh -> qh\", attention_weights, V), attention_weights\n",
        "\n",
        "# get a more natural sentence\n",
        "sentence = \"Masked attention allows implementing dependency constrains between inputs and outputs.\"\n",
        "token_ids = torch.tensor(glove_tokenizer.encode(sentence).ids)\n",
        "tokens = [glove_vocabulary[x] for x in token_ids]\n",
        "vectors = embeddings(token_ids)\n",
        "\n",
        "# EXERCISE: Implement the masks corresponding to each factorization\n",
        "# Hint: check the documentation for `torch.diag()`, `torch.triu()` and `torch.tril()`\n",
        "T = len(token_ids)\n",
        "masks = {\n",
        "    \"left-to-right\": ..., # <- YOUR CODE HERE\n",
        "    \"bidirectional\": ..., # <- YOUR CODE HERE\n",
        "    \"right-to-left\": ..., # <- YOUR CODE HERE\n",
        "}\n",
        "for key in masks.keys():\n",
        "    if masks[key] is not None:\n",
        "        masks[key] = masks[key] = torch.where(masks[key] == 0, 0, -math.inf)\n",
        "\n",
        "# visualized the log of the masked attention map\n",
        "fig, axes = plt.subplots(ncols=1+len(masks), figsize = (16,6), sharex=False, sharey=False, dpi=300)\n",
        "# plot the gradient map from the RNN LM\n",
        "axes.flat[0].imshow(grad_magnitude, sns.color_palette(\"viridis\", as_cmap=True))\n",
        "axes.flat[0].set_xlabel(\"$t$ (input)\")\n",
        "axes.flat[0].set_ylabel(\"$t'$ (output)\")\n",
        "axes.flat[0].grid(False)\n",
        "axes.flat[0].set_title(\"gradient map (RNN LM)\")\n",
        "# plot the attention map\n",
        "for ax, (mask_name, mask) in zip(axes.flat[1:], masks.items()):\n",
        "    if mask is not None:\n",
        "        H, attention_map_masked = masked_attention(vectors, vectors, vectors, mask=mask)\n",
        "        plot_attention_map(attention_map_masked.log(), tokens, tokens, ax=ax, color_bar=False)\n",
        "    ax.set_title(f\"Attention map {mask_name}\")\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "lwuFdsZQuGIO"
      },
      "source": [
        "----\n",
        "## IV. Transformers\n",
        "\n",
        "<img src=\"images/transformer.png\" alt=\"Transformer architecture\" width=\"600\"/>\n",
        "\n",
        "In this section we are going to introduce the [Transformer (\"Attention is all you need\", Vaswani (2017))](https://arxiv.org/abs/1706.03762) architecture.\n",
        "\n",
        "For further information, see the excellent PyTorch tutorial [\"language translation using Transformers\"](https://pytorch.org/tutorials/beginner/translation_transformer.html) and blog article [\"Annotated Transformer\"](https://nlp.seas.harvard.edu/2018/04/03/attention.html), which review the original in great details and provide additional content such as visualizations of the learned attention maps.\n",
        "\n",
        "**Architecture** A Transformer is composed of two main components: a decoder which implements a language model and an encoder. The encoder is only required for conditional language models like those used in translation tasks. Each of the two components is made by stacking Transformer layers (layers with and without conditioning). Each layer transforms a sequence of hidden state $\\mathbf{h}_{1:T}^l$ into another sequence $\\mathbf{h}_{1:T}^{l+1}$. The input tokens are converted into the first state $\\mathbf{h}_{1:T}^0$ using an embedding layer coupled with positioal encodings. the last hidden state $\\mathbf{h}_{1:T}^{L}$ is projected into the vocabulary space using a liner layer."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "h5P9m0_PuGIO"
      },
      "source": [
        "### IV.a Positional encodings\n",
        "\n",
        "The position of a word in the sentence can make a difference in the type of meaning that word is assuming. In order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence. While an RNN see the tokens sequntially and therefore can get an idea of the position of the different words in the sentence, a Transformer model is not processing the input sequentially, therefore this model needs a way to understand which part of the sentence it is processing.\n",
        "\n",
        "The positional encoding should satisy the following properties:\n",
        "- for each time-step, i.e. word position in a sentence, it should produce a unique vector;\n",
        "- the distance between any two time-steps should be consistent;\n",
        "- it should be deterministic.\n",
        "\n",
        "In the paper they proposed to use a positional encoding that produce a vector of size $d_{\\text{model}}$ as the embedding size, so we can add them together. The method proposed by the paper can be defined as:\n",
        "\n",
        "\\begin{align}\n",
        " PE(t,2i) = \\sin{\\left(\\frac{t}{10000^{\\frac{2i}{d_{\\text{model}}}}}\\right)} \\quad\\text{and}\\quad PE(t,2i+1) = \\cos{\\left(\\frac{t}{10000^{\\frac{2i}{d_{\\text{model}}}}}\\right)}\n",
        "\\end{align}\n",
        "\n",
        "where $t$ is the position of the world in the sentence and $i$ is the index in the embedding. To do this we have to consider $i=0,2,4,\\dots$. Indeed, then for each $i$ we compute both the functions above and thertefore $2i$ and $2i+1$ represents the column index of the positional encoding vector. In other words, for each even index $i$ we apply the first equation and for every odd index we apply the second one. Therefore, each dimension of the positional encoding corresponds to a sinusoid and the vector that we will obtain will have the following form:\n",
        "\n",
        "$$ PE = [ \\sin{(t)}, \\cos{(t)}, \\sin{\\left(\\frac{t}{10000^{\\frac{2}{d_{\\text{model}}}}}\\right)}, \\cos{\\left(\\frac{t}{10000^{\\frac{2}{d_{\\text{model}}}}}\\right)}, \\cdots]$$\n",
        "\n",
        "\n",
        "The class defining the Positional Encoding is given by this code snippet. We can also try to plot and look at the positional encoding we get, this is done in the second code snippet."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 300
        },
        "id": "Enprc5RiuGIO",
        "outputId": "f4ddf3d8-3a17-4ec8-be57-61d01f07897c"
      },
      "outputs": [],
      "source": [
        "class PositionalEncoding(nn.Module):\n",
        "    \"Implement the PE function.\"\n",
        "    def __init__(self, d_model, dropout, max_len=5000):\n",
        "        super(PositionalEncoding, self).__init__()\n",
        "        self.dropout = nn.Dropout(p=dropout)\n",
        "\n",
        "        # Compute the positional encodings once in log space.\n",
        "        pe = torch.zeros(max_len, d_model)\n",
        "        position = torch.arange(0, max_len).unsqueeze(1)\n",
        "        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))\n",
        "        pe[:, 0::2] = torch.sin(position * div_term)\n",
        "        pe[:, 1::2] = torch.cos(position * div_term)\n",
        "        pe = pe.unsqueeze(0)\n",
        "        self.register_buffer('pe', pe)\n",
        "\n",
        "    def forward(self, x):\n",
        "        x = x + self.pe[:, :x.size(1)]\n",
        "        return self.dropout(x)\n",
        "\n",
        "# let's assume we havee a sequence of length 10 and embedding of length 128\n",
        "# how their positional encoding looks like\n",
        "\n",
        "plt.figure(figsize=(10, 4))\n",
        "\n",
        "# we define the positional encoding for a 128-dimensional vector and for this \n",
        "# example we do not use dropout\n",
        "pe = PositionalEncoding(64, dropout=0, max_len=1000)\n",
        "\n",
        "# note here we are assuming our input to be\n",
        "# batch_size x sequence_length x d_model\n",
        "# but in pytorch they are assuming \n",
        "# sequence_length x batch_size x d_model\n",
        "# we also assume zeros so we can see how the positional encoding looks like\n",
        "x_inputs = torch.zeros(1, 512, 64)\n",
        "y = pe(x_inputs)\n",
        "\n",
        "sns.heatmap(y.squeeze(0).T, cmap=sns.color_palette(\"viridis\", as_cmap=True))\n",
        "plt.ylabel('Dimension')\n",
        "plt.xlabel('Position in the sentence')\n",
        "plt.gca().invert_yaxis()\n",
        "plt.show()\n",
        "\n",
        "# in the figure below each row corresponds to the vector we are adding \n",
        "# to our embedding vector when the word is at that position in the sentence"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "zoqePMoouGIP"
      },
      "source": [
        "### IV.b Base layers\n",
        "\n",
        "*Figure: (Left) A layer of base Transformer layer with three components: multi-head attention layer, feed-forward layer, and add & norm layer. \n",
        "(Right) Multi-head attention layer.*\n",
        "\n",
        "<img src=\"images/transformer-layer.png\" alt=\"base Transformer layer (without conditioning)\" width=\"200\" style=\"margin-right: 100px\"/>\n",
        "<img src=\"images/multi-head-attention.png\" alt=\"Multi-head attention\" width=\"200\"/>\n",
        "\n",
        "Each base layer of index $l$ takes a sequence of hidden state $\\mathbf{h}_{1:T}^l$ as input, and output another sequence $\\mathbf{h}_{1:T}^{l+1}$.\n",
        "\n",
        "**Multi-head attention** The attention mechanism introduced in the previous section depends on a softmax of inner-products, which might be sparse depending on the value of the vectors, in that case, the layer can only attend to a few positions in the sequence. To enable attending to more positions in the input sentence, multiple attention mechanism can be used in parallel. This is what we call a multi-head attention layer:\n",
        "$$\n",
        "\\begin{align}\n",
        "\\mathrm{MultiHeadAttention} \\left(\\mathbf{Q}^{1:P}, \\mathbf{K^{1:P}}, \\mathbf{V}^{1:P}, \\mathbf{M} \\right) = [ \\mathrm{Attention} \\left(\\mathbf{Q}^{1}, \\mathbf{K^{1}}, \\mathbf{V}^{1}, \\mathbf{P} \\right), \n",
        "\\ldots\n",
        "\\mathrm{Attention} \\left(\\mathbf{Q}^{P}, \\mathbf{K^{P}}, \\mathbf{V}^{P}, \\mathbf{M} \\right)] \\ ,\n",
        "\\end{align}\n",
        "$$\n",
        "where each set of vectors $\\mathbf{Q}^{i}, \\mathbf{Q}^{i}, \\mathbf{Q}^{i}$ corresponding to the head index $i$ is obtained using a separate linear transformation of the input sequence.\n",
        "\n",
        "**Feed-forward** The multi-head attention layer allows looking up multiple positions of the input sequence, but the output is only a linear combination of the value vector $\\mathbf{V}$. A multi-layer neural network (feed-forward layer) is applied **element-wise** to each element of the sequence of hidden states to allow modelling more complex non-linear dependencies.\n",
        "\n",
        "**Add & Norm** A Transformer is a deep neural network, and therefore might be difficult to optimize. Similarly deep neural networks in the image processing field, Transformer layer rely on two stabilizing components:\n",
        "1. [Residual connections](https://arxiv.org/abs/1512.03385) allow to bypass the attention layer as well as the feed-forward layer.\n",
        "2. [Layer normalization](https://arxiv.org/abs/1607.06450): allow enforcing that the output of a Transformer layer has values that are properly distributed\n",
        "\n",
        "**Implementation:** Here is an implementation for the attention and multi-head attention. You can also check the official [PyTorch implementation](https://pytorch.org/docs/stable/_modules/torch/nn/modules/activation.html#MultiheadAttention)."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "PDOhBgQSuGIP"
      },
      "outputs": [],
      "source": [
        "def attention(query, key, value, mask=None, dropout=None):\n",
        "    \"Compute 'Scaled Dot Product Attention'\"\n",
        "    d_k = query.size(-1)\n",
        "    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)\n",
        "    if mask is not None:\n",
        "        scores = scores.masked_fill(mask == 0, -math.inf)\n",
        "    p_attn = F.softmax(scores, dim = -1)\n",
        "    if dropout is not None:\n",
        "        p_attn = dropout(p_attn)\n",
        "    return torch.matmul(p_attn, value), p_attn"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "8Z_vEKJFuGIQ"
      },
      "outputs": [],
      "source": [
        "class MultiHeadedAttention(nn.Module):\n",
        "    \"\"\"A simple Multi-head attention layer.\"\"\"\n",
        "    def __init__(self, h, d_model, dropout=0.1):\n",
        "        \"Take in model size and number of heads.\"\n",
        "        super(MultiHeadedAttention, self).__init__()\n",
        "        assert d_model % h == 0\n",
        "        # We assume d_v always equals d_k\n",
        "        self.d_k = d_model // h\n",
        "        self.h = h\n",
        "        self.linears = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(4)])\n",
        "        self.attn = None # store the attention maps\n",
        "        self.dropout = nn.Dropout(p=dropout)\n",
        "\n",
        "    def forward(self, query, key, value, mask=None):\n",
        "        nbatches = query.size(0)\n",
        "        if mask is not None:\n",
        "            # Same mask applied to all h heads.\n",
        "            mask = mask.unsqueeze(1)\n",
        "\n",
        "        # 1) Do all the linear projections in batch from d_model => h x d_k \n",
        "        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (query, key, value))]\n",
        "\n",
        "        # 2) Apply attention on all the projected vectors in batch. \n",
        "        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)\n",
        "\n",
        "        # 3) \"Concat\" using a view and apply a final linear. \n",
        "        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)\n",
        "        return self.linears[-1](x)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "NZiGQ6I3uGIQ"
      },
      "source": [
        "### IV.c Conditional layers\n",
        "\n",
        "In the description of the base layer, we have applied attention to the input sequence (*self-attention*). Machine translation is a sequence-to-sequence task, which requires and *encoder* component, that encodes a source text into a sequence of hidden states $\\mathbf{g_{1:T'}}$. The base layer can be modified with an additional attention layer that is conditionned on the source text. Given a sequence of hidden states $\\mathbf{h}_{1:T}$, the conditional attention layer is:\n",
        "$$\n",
        "\\mathrm{Attention}(\\mathbf{Q}(\\mathbf{h}_{1:T}), \\mathbf{K}(\\mathbf{g}_{1:T'}), \\mathbf{V}(\\mathbf{g}_{1:T'})) \\ .\n",
        "$$\n",
        "Conditional attention layers are place right after the self-attention layers, before the feed-forward layer (see diagram).\n",
        "\n",
        "### IV.d Pre-training as language models\n",
        "\n",
        "Transformers (the decoder component) are language models can therefore be pre-train on vast amount of unlabelled text via maximum likelihood or pseudo likelihood. They allow obtaining contextualized text representations, that can be applied to a multitude of downstream tasks (question-answering, classification, ...). While the original Transformer architecture was applied to a sequence-to-sequence problem with a component that encodes the source text and a language model decoder conditioned on the encoded source text. Transformer-based language models consist in a single decoder component (without conditional attention). The two main alternatives for language modelling are:\n",
        "\n",
        "* [Generative Pre-trained Transformers (GPT)](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf): autoregressive left-to-right language models implemented using a Transformer.\n",
        "\n",
        "* [Bidirectional Encoder Representations from Transformers (BERT)](https://arxiv.org/abs/1810.04805): a masked language model trained to predict tokens that are randomly masked out (masked language model) and trained to predict whether two sentences are related or not (next-sentence prediction (NSP) task)."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "IJhmp8kJuGIQ"
      },
      "source": [
        "___\n",
        "## V. Applications of Transformer-based language models\n",
        "\n",
        "Let's experiment with a few pre-trained Transformers using the [🤗 HuggingFace](https://huggingface.co/) environment and the [OpenAI API](https://beta.openai.com/docs/guides/completion). \n",
        "\n",
        "HuggingFace is a company that manages multiple open source projects:\n",
        "* **[tokenizers](https://huggingface.co/docs/tokenizers/index)**: a very efficient implementation of tokenizers using Rust.\n",
        "* **[datasets](https://huggingface.co/docs/datasets/index)**:a library for handling and sharing large datasets (in particular they use [Apache Arrow](https://arrow.apache.org/)  for efficient data loading from disk)\n",
        "* **[transformers](https://huggingface.co/docs/transformers/index)**: implementation and sharing of Transformers in Tensorflow, PyTorch and JAX\n",
        "\n",
        "All HuggingFace models and datasets (text, audio, image and more) can be accessed through the [🤗 Hub](https://huggingface.co/), and many models can tested lives on [🤗 spaces](https://huggingface.co/spaces). In the examples bellow, we will first try to manipulate data and models using lower primitives (tokenizing data, loading a model, generating / inference), so you can interact if the intermediate variables if you want to. Then we will us the blackbox [`Pipeline`](https://huggingface.co/docs/transformers/v4.22.1/en/main_classes/pipelines#transformers.pipeline) object. If you want to apply Transformers without modifying any of the components, the `Pipeline` can be used to perform complex tasks in one line of code, as showed here with the translation task.\n",
        "\n",
        "The OpenAI API gives access to GPT-3 ([\"Language Models are Few-Shot Learners\", Brown et al. (2020)](https://arxiv.org/abs/2005.14165)) through a [playground](https://beta.openai.com/playground), where you can test the text completion capabilities of these models. GPT-3 is a large language model (up to 175 billion parameters) which has acquired impressive language understanding capabilities. It can be applied to solve new tasks without task-specific fine-tuning. [OpenAI gives you $18 to of API credits, but careful with the number of calls: running the largest version of GPT-3 (´davinci´) can be expensive](https://openai.com/api/pricing/).\n",
        "\n",
        "### V.a Language generation\n",
        "\n",
        "Let's experiment with GPT-2 (in the notebook, we use the smaller [`distilgpt2`](https://huggingface.co/distilgpt2), but feel free to use the original `gpt2` if you have enough compute)\n",
        "\n",
        "**Experiment** Generate text using GPT-2:"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 49,
          "referenced_widgets": [
            "cfec14d18e554e998de7ce8cc71496f8",
            "21332697cbd5478e9a2158302f71ed84",
            "4165d6cf31964bc9bad2e555153a6c94",
            "c2de5f63d1654c869877718fad50265c",
            "5013660e1a0149e1b228eb15ed544e5c",
            "6bd38346f1fd4246b5f8f6aad5176773",
            "28864b35a0614bc890d346660683629e",
            "8d3dc00e3e774bd99e52e494f0a9818a",
            "b1bed1525f1b4abebc507eb9f3ae1ba7",
            "de5cb07d322c4bb4ac96700939e98dae",
            "ee2cbeb2ea6749658bfb764f181c115f"
          ]
        },
        "id": "axCm-cueuGIR",
        "outputId": "39b35f18-92b1-49f6-f138-5d5b6e4b9625"
      },
      "outputs": [],
      "source": [
        "# load a GPT-2 model\n",
        "model_id = \"distilgpt2\"\n",
        "# model_id = \"gpt2\"\n",
        "tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)\n",
        "model = transformers.AutoModel.from_pretrained(model_id)\n",
        "# Here we want to load the weights of GPT2 as an autoregressive LM, or \"causal\" LM: we use the class `AutoModelForCausalLM`.\n",
        "model = transformers.AutoModelForCausalLM.from_pretrained(model_id)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "uug5lB4auGIR",
        "outputId": "210e8af9-4fdf-46d9-ee67-c613fbda92b8"
      },
      "outputs": [],
      "source": [
        "# encode context the generation is conditioned on\n",
        "prompt = tokenizer.bos_token # use the Begining Of Sentence token to initialize allow generating text from scratch\n",
        "input_ids = tokenizer.encode(prompt, return_tensors='pt')\n",
        "# expand input_ids, so we can sample multiple generations in parallel\n",
        "input_ids = input_ids.repeat(5, 1)\n",
        "\n",
        "# generate text until the output length (which includes the context length) reaches 50\n",
        "# documentation for `model.generate()` https://huggingface.co/docs/transformers/v4.22.1/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate\n",
        "output = model.generate(input_ids, do_sample=True, temperature=1., max_length=50)\n",
        "decoded = tokenizer.batch_decode(output)\n",
        "for txt in decoded:\n",
        "    txt = txt.replace(\"\\n\", \"\")\n",
        "    print(txt)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "pBJNlOKRuGIR"
      },
      "source": [
        "**Exercise 9**: Play with the temperature parameter, what is it controlling? how can you related it to the definition of a language model?\n",
        "\n",
        "> *Insert your your answer here*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "mxf9whBIuGIS"
      },
      "source": [
        "### V.b. Prompt-based learning\n",
        "\n",
        "Prompt-based learning consists in framing a problem into a *prompt* which completion by a language model corresponds to the answer. In other words, it consists in using natural language to interface with language models.\n",
        "\n",
        "**Experiment** Do language models know what hygge is?\n",
        "\n",
        "**Exercise 10**: Write a prompt that triggers a language model to define \"hygge\".\n",
        "\n",
        "> *write your prompt here and test it using GPT-2 and GPT-3*"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "EyOWrcVIuGIS",
        "outputId": "c3f8ba75-e2f5-4077-b1f7-79e0b5adc3d8"
      },
      "outputs": [],
      "source": [
        "prompt = \"Q: What is Hygge? A:\"\n",
        "input_ids = tokenizer.encode(prompt, return_tensors='pt')\n",
        "input_ids = input_ids.repeat(5, 1)\n",
        "output = model.generate(input_ids, do_sample=True, temperature=1, max_length=50)\n",
        "decoded = tokenizer.batch_decode(output)\n",
        "for txt in decoded:\n",
        "    txt = txt.replace(\"\\n\", \"\")\n",
        "    print(txt)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "q4j2-kvvuGIS"
      },
      "source": [
        "**Exercise 11**: Test your prompt [with GPT-3](https://beta.openai.com/playground)\n",
        "\n",
        "> *Insert your prompt and the GPT-3 completion here*\n",
        "\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "y9jvbMrWtKcj"
      },
      "source": [
        "\n",
        "**More prompt-based learning** Large language models can answer complex questions. In [\"Can large language models reason about medical questions? Liévin et al. (2022)\"](https://arxiv.org/abs/2207.08143), we observed that GPT-3 can in many cases solve complex medical problems which require 1) reading and understaing the provided medical cases, 2) retrieving expert knowledge (implicitely stored in the weights) and 3) detail a multi-step reasoning to reach a conclusion. Multi-step reasoning can be triggered using zero-shot chain-of-thought prompting ([\"Large Language Models are Zero-Shot Reasoners\", Kojima et al. (2022)](https://arxiv.org/abs/2205.11916)), which boils down to prompting a language model with \"*Let's think step by step*\". \n",
        "\n",
        "You can experiment with an example of chain-of-thought prompt using [this template](https://beta.openai.com/playground/p/ka3F37wLN57m62YkYjvyDx0N?model=text-davinci-002)."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "y-dT_fV-uGIS"
      },
      "source": [
        "### V.c Language Translation\n",
        "\n",
        "**Experiment** Let's use the `Pipeline` object to translate French to English in one line of code:"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 227,
          "referenced_widgets": [
            "ac0350f2087a4d74b3dee0de2400545f",
            "378dd87d8d1e499880a8ce6e6ff5f6ef",
            "e799ed767acd4c5f8825bb1bbf7f7bda",
            "feed7ab81f894f04b4e485f54ad25ff0",
            "0ff7be06dab04b83b1bbc27f67f1bae6",
            "eea3ada87a1942458ae8bac1095f0729",
            "466c9e5977b7463b84a6a4d91458bc6d",
            "7431585ef62e4e26a39006d3fd8aec8c",
            "ec5c607f95b3451794e8e3f6ee2a53d5",
            "2491724422214716972d8198d78dba2e",
            "01ae48366913491c910b0587be32596c",
            "aeefe750041e456c8294fa00d2866cdb",
            "ab16e2668a6647dab78c2669e945917d",
            "1480d86a28d842dbaec31ca26b3486fc",
            "b6e133ea752b46b987eadac7d996108f",
            "55ddf3f2dca4458ca9866c4600aa1186",
            "58c2c3c4502940a38c6f842850f43b08",
            "23044cf6a4d04064b4cb3dbf46e22041",
            "4b00c04c8bf34288b10505c897f6516f",
            "e24895da040e4f07ad88ded9086156b1",
            "5a4c991a4e2247e6818707680d47a560",
            "76932440413a4331b2f4754b7b226963",
            "5fe53cfebe1f49fa9b16d3f6183ef62c",
            "a9a3dd627afd4247b8f67d9775489c37",
            "a1818b02b2c44451a27295e72d3147b6",
            "249fb46f6f33468d8a40e76a76371e63",
            "2dab7eeda51541a98b42f291541dde89",
            "335b62d41c464d3a8cd202471a0154d7",
            "aac253fca0774d0db2c8f27ca6f24bca",
            "e54c1368445541539af7f188bc6196cb",
            "521aff2ec667482496021dda759f58d3",
            "8fd8aa6532e24643b264157c63463471",
            "b884187aaa5844bdad69287202c10509",
            "50549cabedf24e11b3bb689bb32b4e47",
            "42b2c0fc94964d76b2c0d606bf26c891",
            "5faeb1b875c341f5863f170005a6858c",
            "91665c2c2aa64904b5ca2b50c82470bb",
            "e84414f3f9a2410c8983e332bed42526",
            "59864ed939c64473ad1da8ed9568304b",
            "9d734fd174764c0492ca1b7593edced2",
            "d1da106abef7438891a93504e436140c",
            "fa11c85885b34c0c8e105c3cec6fbb83",
            "c2ca862568404b5c82a6170dbd90c437",
            "521278900d8241088363e811991c1e66",
            "bb474e5c83f847078224674ef58ab55d",
            "7b9c932216c24b61b1f8c9d3fc315863",
            "97195ac813af429084132711525e4a91",
            "8fc5b50cd6b045a7a131a94d44a67c4a",
            "ac6217065b744df9a9ee700a3f94f655",
            "5b952890c4ac48ed8fd729ed1322a6b8",
            "37643f4536624f2b93c5ef1028de495d",
            "37731c0ff1954df5893b00dcb7f0b6d9",
            "c6c2e069da7a41e583bc0834a54a9b78",
            "8d85f1f723d340d19fb50105344bab69",
            "70ae5680a40a4e4fb7f71c6f951f1479",
            "bc4291a502d54678bfe7fde6b1c51d33",
            "36e0dc35ec09429b8c61529dcbf88cae",
            "a41948d2db514c168bb2801bb7c539d8",
            "cd66da55631c42b3a75eb44b72b06ac2",
            "00a90b713cd14806b103195a3d71b091",
            "735a419b8c944ac9806a9c009dc6a09f",
            "a7bf4c1831c54aa298d2cd3450ed66fa",
            "8c7334d274ee4d3c9d58a0cc39838984",
            "ca99d452a20643fd996ff0b9064b3e1f",
            "7fcf5fa6a46849968f92fd00b2670543",
            "ea5c3ecdef4f46509e6dc204abee8184"
          ]
        },
        "id": "5jV1Xy5SuGIT",
        "outputId": "73e4ae38-afa3-400e-e3b9-76f3ab6f2f51"
      },
      "outputs": [],
      "source": [
        "fr_en_translator = transformers.pipeline(\"translation_fr_to_en\", model=\"Helsinki-NLP/opus-mt-fr-en\")\n",
        "output = fr_en_translator([\"Mon cheval est très rapide\", \"Omelette du fromage\"])\n",
        "rich.print(output)\n",
        "# del fr_en_translator"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "sQKU7HkFuGIT"
      },
      "source": [
        "### V.d Information Retrieval\n",
        "\n",
        "BERT allows converting pieces of text (sentences, paragraphs or even documents) into vectors. [DPR (\"Dense Passage Retrieval for Open-Domain Question Answering\", Karpukhin et al. (2020))](https://arxiv.org/abs/2004.04906) is a BERT model fine-tuned for the task of information retrieval. DPR converts documents and questions into vectors fixed-size vectors which can be compared via inner product. It is then possible to index large collections of documents (Wikipedia) efficiently using maximum inner product search libraries like [`faiss`](https://faiss.ai).\n",
        "\n",
        "**Experiment** Let's implement a simple search engine using a pre-trained DPR. In the same vein as the previous experiments, we define each document as a short factual sentences \"*`<city>` is the capital city of `<country>`*\". The questions are set to \"*What is the capital city of `<country>`?*\"."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 177,
          "referenced_widgets": [
            "36247c8d56a24a3187a71161fc877ef5",
            "c110d11390cd4e9ca915af693a681c4a",
            "23e3e9d40ef949d6b2b7df7e37e5ac04",
            "f8eaece4960948eebf0dafe087eb252e",
            "17ba3f0055114be6bbf927e6bd181d2a",
            "fe48ae258e904d6aaac09c728868ce41",
            "adf45441dafd499bb5eab3b3a969a3c0",
            "e86fce17da1142cf892cad26a42c5610",
            "3ac1b5e8c3764a85b55d2df5a5890eed",
            "87349896035b48a99c10939bc954222b",
            "0c9423eb6df1485e94e4cbee5e545df4",
            "71c9f44ab15048879ce62315ff0c48fb",
            "ee1d498e7e20480988712721acd42b12",
            "5e5b7d74bc1545bcbf3f2cc61edb660b",
            "18033c64e10e4f31b663e47d70bc8fe2",
            "384f16c5e1d54c2397d9c75d06c1bfbd",
            "8cbd54818cbf49e7abe5a9b404e30320",
            "b2990a41d73b4f058a1cf336e78b9aea",
            "28aaceda6c194ab09a986b360e9f1d43",
            "f4d7b4451ac64e9e9cba9c79a2a5ed58",
            "99233039049945f8b9db2d9f13e084e6",
            "9964fe9b3e0f464ba4de95aeefc75422",
            "49b9d7f7b9e645ed8273bc1fbbd1bbf5",
            "87b0156a2a4a4f27bb6c2232c29b8723",
            "69b9496a0d6d44fdaa43b7e5a607c407",
            "eea82cd791c141f799b256e9023eb0c2",
            "9da3c4afad1b441386ad7f10b32d5db9",
            "707373ef81ca460d88b754c71800b72c",
            "068453e620e34f0cb09aa4cd2a9043c6",
            "7656cd9f368f4469a4c7f410e5977ec2",
            "6c9e1f3b770b429bb6ee5ef7234dd78f",
            "645f9c5a7dc84530a409a75b0373c283",
            "b1a60018182448fc9ca922efed6a4ce4",
            "a1d20c56010243e38917d595f6514253",
            "097505d47ab44732968670d7971503e4",
            "b0de99cc03ce43bdba8108442f8db6f3",
            "f272c61ecafb44299bcc8974f6062dba",
            "1a9aca81153942569a0dbccb93555619",
            "3bfa7d3b037141758002c4b222db6aa8",
            "45ba9ea28ab942a0a46bb51abc2d51d6",
            "f62aa191dfa443678339b194a59d75ff",
            "f20fabbf6f064b8a8be0c99065f707b4",
            "2f2bbcf674964943ae306aa8d1c6bb3e",
            "ac6aaa3c18cd4824971c422068858a53",
            "ba4f66d47deb45e28d559085f3eff4ae",
            "67ce1f7d87c34f4dafdcd908c7dd7b84",
            "6c6139aefafc430b9ee6ce8644399eda",
            "8c15f48271214e97b30e3f5ad1658f72",
            "64413aa525174ffc90e7bed22b532b7b",
            "f8bb3d12d0144d72b7f5bed4055d2b29",
            "dc775613d7d74f4a84f0731f595eaba2",
            "5096a37007de48ab8a6f3dd6b3a9e68f",
            "4b356a43e7784e1e86b250a0aa5e4f8c",
            "9618539b7cdb432d8e374aa593ad7385",
            "d1b47ba0a7e84c3989a232e2e8532f9c"
          ]
        },
        "id": "EjbZ-qd4uGIT",
        "outputId": "5971c006-311d-4556-956f-f7da689ba69f"
      },
      "outputs": [],
      "source": [
        "model_id = \"facebook/dpr-ctx_encoder-single-nq-base\"\n",
        "tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)\n",
        "model = transformers.AutoModel.from_pretrained(model_id).eval()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "ygptaSOmuGIU",
        "outputId": "6a91cc20-af8a-49c0-9e58-f9d2ea7cdfab"
      },
      "outputs": [],
      "source": [
        "# Define some documents and questions\n",
        "documents = [f\"{city} is the capital of {country}\" for city, country in zip(keys_cities_labels, keys_labels)]\n",
        "questions =  [f\"What is the capital city of {country}?\" for country in keys_cities_labels]\n",
        "\n",
        "# Tokenizer\n",
        "tokenizer_args = {\"padding\":True, \"truncation\":True, \"return_tensors\": \"pt\"}\n",
        "documents_ = tokenizer(documents, **tokenizer_args)\n",
        "questions_ = tokenizer(questions, **tokenizer_args)\n",
        "\n",
        "# Compute the question and document vectors\n",
        "with torch.no_grad():\n",
        "    document_vectors = model(**documents_).pooler_output\n",
        "    question_vectors = model(**questions_).pooler_output\n",
        "\n",
        "# compute the inner product (attention map) between all pairs of questions and documents\n",
        "attention_map = question_vectors @ document_vectors.T\n",
        "attention_map = attention_map.softmax(dim=1)\n",
        "\n",
        "# plot as an attention map\n",
        "plot_attention_map(attention_map, questions, documents)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ail6ypmsFdO8"
      },
      "source": [
        "**Exercise 12**: Is DPR retrieving the right document for each question? Comment on the shape of the attention map.\n",
        "\n",
        "> *Insert your answer here*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "lGRt_VQDuGIU"
      },
      "source": [
        "---\n",
        "## Credits and additional references\n",
        "\n",
        "### Credits\n",
        "\n",
        "Written by Valentin Liévin and reusing parts of the [Notebook](https://github.com/DeepLearningDTU/Deep-Learning-with-PyTorch-1weekcourse/tree/master/3_Recurrent/notebook_transformer) from Frederico Bergamin.\n",
        "\n",
        "\n",
        "### Additional references\n",
        "\n",
        "1. RNNs.\n",
        "- [Blog post: \"The Unreasonable Effectiveness of Recurrent Neural Networks\" (Karpathy)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)\n",
        "- [lecture: \"Language Models and RNNs\" (Stanford)](https://www.youtube.com/watch?v=iWea12EAu6U&list=PLoROMvodv4rOhcuXMZkNm7j3fVwBBY42z).\n",
        "\n",
        "2. Transformers\n",
        "- [Video from the main author of the paper](https://www.youtube.com/watch?v=rBCqOTEfxvg&t=894s)\n",
        "- [Video lecture from Deepmind](https://www.youtube.com/watch?v=8zAP2qWAsKg&t=2073s)\n",
        "- [Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)\n",
        "- [Blog Post on the Positional Encoder](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)\n",
        "- [Transformer from Scratch](https://e2eml.school/transformers.html#positional_encoding)\n",
        "- [Another blog post explaining transformer](https://jalammar.github.io/illustrated-transformer/)\n",
        "\n",
        "### Re-using the images\n",
        "\n",
        "[Figma design file](https://www.figma.com/file/d7BHs0yMhM9CTer43mrTe5/deep-learning-course?node-id=0%3A1)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ib3AVsyguGIU"
      },
      "source": []
    }
  ],
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "collapsed_sections": [],
      "provenance": []
    },
    "interpreter": {
      "hash": "e72cf85b1cff461a3287147b79b2cfd4bf5f57e721dfc08300645559174c8f4b"
    },
    "kernelspec": {
      "display_name": "Python 3.9.13 ('jupyter')",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.9.13"
    },
    "orig_nbformat": 4,
    "widgets": {
      "application/vnd.jupyter.widget-state+json": {
        "00046c6611c940058bac1aa75da16f95": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "00a90b713cd14806b103195a3d71b091": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "00f958e144f346cc8ff25825d293f1d3": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": "hidden",
            "width": null
          }
        },
        "01ae48366913491c910b0587be32596c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "0330f9b958f34c46a34e62b47568fa77": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "04a785e5695545f8905306feee92b618": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "068453e620e34f0cb09aa4cd2a9043c6": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "06a4618bda4949f2ace26450b1d8052e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_89d19eb4b1a243bf8318b037f3eb1eb2",
            "max": 29,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_4a41a1e705b549f8b094e5a1432ea389",
            "value": 29
          }
        },
        "0869911afc8549d9bd4ce0e7f97bdc0c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_4bde8be94bb945b7869e4fa261d686dd",
            "max": 213450,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_9b6c07e98b4c4e95bdddc7dac23d69cf",
            "value": 213450
          }
        },
        "08995f7f72f14dd8a22565ee4ae41215": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "091851ddeeb6437aa1a8cc8fbe2bc17c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_ddb0ac228105475a83b334351aed3439",
            "placeholder": "​",
            "style": "IPY_MODEL_a642a8054e0a4ba08a516b18498ee769",
            "value": "Downloading metadata: 100%"
          }
        },
        "097505d47ab44732968670d7971503e4": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3bfa7d3b037141758002c4b222db6aa8",
            "placeholder": "​",
            "style": "IPY_MODEL_45ba9ea28ab942a0a46bb51abc2d51d6",
            "value": "Downloading: 100%"
          }
        },
        "09b9bc4a5757400fb352de17b2eb1cab": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "09e77e9a34f24fedbdb90b2112f27347": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "0c9423eb6df1485e94e4cbee5e545df4": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "0f938c22289d45d9bdd7929ed1a97626": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_67e91a4c9376417dbb9916e83f5ccdd4",
            "placeholder": "​",
            "style": "IPY_MODEL_9ca45bbcbdbe4eab98cb67ed1249d8fb",
            "value": "Downloading: 100%"
          }
        },
        "0ff7be06dab04b83b1bbc27f67f1bae6": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "135d819a8d7f4e3ba85bed56b318a0d3": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "1480d86a28d842dbaec31ca26b3486fc": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_4b00c04c8bf34288b10505c897f6516f",
            "max": 300827685,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_e24895da040e4f07ad88ded9086156b1",
            "value": 300827685
          }
        },
        "148fd6dd13ca40199c14bc396a741d83": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "15280581483b47f18aa57296f25d8334": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "15b3668ae54c4fb88b9d67d5756122c4": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "16ebe063385841faba7f79ae57dad474": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "1732733c760947fb920e9de7606705ef": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_7e7492861d2d4334aaa522829fe52d0a",
              "IPY_MODEL_5a37cba6a0da4612bbcc177d0d547d31",
              "IPY_MODEL_1838ebcc7ea34641938901ab9415cf58"
            ],
            "layout": "IPY_MODEL_00f958e144f346cc8ff25825d293f1d3"
          }
        },
        "17ba3f0055114be6bbf927e6bd181d2a": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "17d2e815f3ca46fa82167ccd1eb85b2a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "18033c64e10e4f31b663e47d70bc8fe2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_99233039049945f8b9db2d9f13e084e6",
            "placeholder": "​",
            "style": "IPY_MODEL_9964fe9b3e0f464ba4de95aeefc75422",
            "value": " 492/492 [00:00&lt;00:00, 13.1kB/s]"
          }
        },
        "1838ebcc7ea34641938901ab9415cf58": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_255c7719289d40ee8d7858e7c97f62b8",
            "placeholder": "​",
            "style": "IPY_MODEL_d4447eac662145188017fe281566cfea",
            "value": " 119165/120000 [00:04&lt;00:00, 28624.98 examples/s]"
          }
        },
        "1a1d215fecb049429b7cac703cbb010d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_238bca108c114c5bb1bb20c6a9126bee",
              "IPY_MODEL_5135f2da12774e49b3941a75ee539fb0",
              "IPY_MODEL_d1e2d720723b4d99bda65860a15ad427"
            ],
            "layout": "IPY_MODEL_298186b7e41e4c3c9047ed35d8ca5b79"
          }
        },
        "1a511d9f81cd434ea9a4a0baa1ad43f2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5b85631ac5a446e5884020fb7ba4b96f",
            "placeholder": "​",
            "style": "IPY_MODEL_fc44ba4fc74f472a95cbeab1ea179b89",
            "value": " 1.36M/1.36M [00:01&lt;00:00, 1.03MB/s]"
          }
        },
        "1a9aca81153942569a0dbccb93555619": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "1b32d1e2681d4561840a29c55d11bc9f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "1ba2f6eb67f342c4aca41c7b8624ddde": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "1c28b4c4a1bd44ddbed1028e3a755bb5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "1d9d2cf9cd59425497adf32e4dd59261": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_9b601f47611b4281b2b2d1fcab39c38b",
            "max": 570,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_670ce1a5a4a3477e970db3210192ec4a",
            "value": 570
          }
        },
        "1de84e67a3b34cddb1c73beae7888833": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "1e9772f564fa4d4390518664a1595197": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f34c4b62a07949b68758d6f6ffd1b31f",
            "placeholder": "​",
            "style": "IPY_MODEL_7ce42569074943d3af0ca12b2ac351d7",
            "value": " 698/698 [00:00&lt;00:00, 20.9kB/s]"
          }
        },
        "2066c6d9fe2a4d6f876e81fbbf64e653": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_00046c6611c940058bac1aa75da16f95",
            "placeholder": "​",
            "style": "IPY_MODEL_08995f7f72f14dd8a22565ee4ae41215",
            "value": " 466k/466k [00:00&lt;00:00, 310kB/s]"
          }
        },
        "20d9a77f209d473bb1d2e751d706913e": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "21332697cbd5478e9a2158302f71ed84": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_6bd38346f1fd4246b5f8f6aad5176773",
            "placeholder": "​",
            "style": "IPY_MODEL_28864b35a0614bc890d346660683629e",
            "value": "Downloading: 100%"
          }
        },
        "21e3b941f0b24664895ad9913206d11c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "23044cf6a4d04064b4cb3dbf46e22041": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "233c55aac32242f6be5085631989f7c8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_586aa53e10754cb8948e3636da7aef9c",
            "placeholder": "​",
            "style": "IPY_MODEL_e0ea063acfe245c783f720279a8004cd",
            "value": " 862M/862M [00:48&lt;00:00, 17.8MB/s]"
          }
        },
        "238bca108c114c5bb1bb20c6a9126bee": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_ca79683bd37243f98b5f10f2e6f373c4",
            "placeholder": "​",
            "style": "IPY_MODEL_c923a57cac3a44389ad8b8bf0920a8aa",
            "value": "#1: 100%"
          }
        },
        "23e3e9d40ef949d6b2b7df7e37e5ac04": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_e86fce17da1142cf892cad26a42c5610",
            "max": 28,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_3ac1b5e8c3764a85b55d2df5a5890eed",
            "value": 28
          }
        },
        "242c86c1e834490897262daa0ef5dfcd": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_971ee1ec3e25467fa861cf30207f25a7",
            "placeholder": "​",
            "style": "IPY_MODEL_15b3668ae54c4fb88b9d67d5756122c4",
            "value": " 213k/213k [00:00&lt;00:00, 258kB/s]"
          }
        },
        "2491724422214716972d8198d78dba2e": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "249fb46f6f33468d8a40e76a76371e63": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8fd8aa6532e24643b264157c63463471",
            "placeholder": "​",
            "style": "IPY_MODEL_b884187aaa5844bdad69287202c10509",
            "value": " 42.0/42.0 [00:00&lt;00:00, 1.32kB/s]"
          }
        },
        "255c7719289d40ee8d7858e7c97f62b8": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "28864b35a0614bc890d346660683629e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "28aaceda6c194ab09a986b360e9f1d43": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "28ac8463e3f14e899b7e16553d991d41": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "298186b7e41e4c3c9047ed35d8ca5b79": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2af41c4c23e24e3db73a9003a32d0097": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2ba33d1e19ac48149fc2c78e1baba2a9": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "2bd6ec97d073442eb08d141c07fd199d": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2cb7d45a1a804c109e67e9a6c50976ee": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2d5966ea44ec4e469a758037f90aaa3b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_527265f507a946a283f7437e679fa909",
            "placeholder": "​",
            "style": "IPY_MODEL_a10e589c393744f58fc09cc1b97c8df5",
            "value": " 380/380 [00:01&lt;00:00, 324.76ba/s]"
          }
        },
        "2d966f4595464c49be7f5b323cf54b1c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2da10880d1da4b7eba255333d171d653": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5cb53426cc29476dadb4c2fb88c5b5ad",
            "placeholder": "​",
            "style": "IPY_MODEL_3ce1683b100b4e249a4272220b0b6bed",
            "value": "Downloading: 100%"
          }
        },
        "2dab7eeda51541a98b42f291541dde89": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2de4643e661d448caaa7a1a7e660c6b0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "2f2bbcf674964943ae306aa8d1c6bb3e": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "2f2fad7073e347e28b71cc61ecb53513": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_da110a0f535848838ac3018c9bd5603c",
              "IPY_MODEL_ad3776cc4d204d2daaa0bc18c24bdcba",
              "IPY_MODEL_38efabdd23f74921951c71ae079405d7"
            ],
            "layout": "IPY_MODEL_9ca3d1acbf904aa2b7f9f1f1475a5e9f"
          }
        },
        "2fa2d24e07cf4c4cba4cd296ba9281b3": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "30caa70b8e2b4ca28af34fb14cd0cf86": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_eb07e619043940ecb3dd33ec3f38b03c",
            "max": 862182753,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_d7522bc3874c4285b33ca65febd3f2d5",
            "value": 862182753
          }
        },
        "30df52fae0f74e1caa14dab8f4689aff": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "31949d7fd38f4c20b8bb734dbf9aaa3b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_636cfa48a2c647efa9ada29b42c37a35",
              "IPY_MODEL_bc150147b2c845f3aead1c4b1fa09c61",
              "IPY_MODEL_a0f74a508afc44ed9c618e1d181807a5"
            ],
            "layout": "IPY_MODEL_689de7097be44956ac7d9a5736b8e5bb"
          }
        },
        "31e124e44206472ab49233418434b9ce": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_8d664b21ce974899bb6a7aaaea21a69e",
              "IPY_MODEL_e2036efdcaab46719ab2b57baf01e2a2",
              "IPY_MODEL_1a511d9f81cd434ea9a4a0baa1ad43f2"
            ],
            "layout": "IPY_MODEL_acfe14fcd0fc484b957fe97cfcb34c4c"
          }
        },
        "335a51f7dd5548c3947a6288495c1737": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_c278bbf65b6d43c0b82af9a3cab5ef9e",
            "placeholder": "​",
            "style": "IPY_MODEL_3f549e3d727043358247df4eddcc7c07",
            "value": " 0/0 [00:00&lt;?, ?it/s]"
          }
        },
        "335b62d41c464d3a8cd202471a0154d7": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3369c01acc964d9b8b5738e59b05b6b4": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "33d93bd8482841f5ad2bc99317c85c27": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_7b964fbfd635438f96ff1858ca33c0e5",
            "placeholder": "​",
            "style": "IPY_MODEL_49d1ba4b752145f4ac0668dfdb4b40b0",
            "value": " 2.65k/2.65k [00:00&lt;00:00, 92.4kB/s]"
          }
        },
        "340c96ed160c42b191291545dfe6848c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "3586240e502f4255ab0d3d66e79b449e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_588dfe8a747a4cd2a9ca35bba8898808",
            "placeholder": "​",
            "style": "IPY_MODEL_6d3fa316dfa24d20bda288ddfb71b4c8",
            "value": " 1.04M/1.04M [00:01&lt;00:00, 961kB/s]"
          }
        },
        "35d7392e1f7441269689a5260050bf8d": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "36247c8d56a24a3187a71161fc877ef5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_c110d11390cd4e9ca915af693a681c4a",
              "IPY_MODEL_23e3e9d40ef949d6b2b7df7e37e5ac04",
              "IPY_MODEL_f8eaece4960948eebf0dafe087eb252e"
            ],
            "layout": "IPY_MODEL_17ba3f0055114be6bbf927e6bd181d2a"
          }
        },
        "36a31d0fa707478285f094582fb2b090": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_b2966a02f9b74b7895c525b6fb21597d",
            "max": 435797,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_67eb4a8763f648c1a4934e076c64bb2b",
            "value": 435797
          }
        },
        "36e0dc35ec09429b8c61529dcbf88cae": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_735a419b8c944ac9806a9c009dc6a09f",
            "placeholder": "​",
            "style": "IPY_MODEL_a7bf4c1831c54aa298d2cd3450ed66fa",
            "value": "Downloading: 100%"
          }
        },
        "37643f4536624f2b93c5ef1028de495d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "37731c0ff1954df5893b00dcb7f0b6d9": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "378dd87d8d1e499880a8ce6e6ff5f6ef": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_eea3ada87a1942458ae8bac1095f0729",
            "placeholder": "​",
            "style": "IPY_MODEL_466c9e5977b7463b84a6a4d91458bc6d",
            "value": "Downloading: 100%"
          }
        },
        "380c55c2680a4f85a81cfb266eace834": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3828d19ebc43415f8ff87568418ec9b2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3369c01acc964d9b8b5738e59b05b6b4",
            "placeholder": "​",
            "style": "IPY_MODEL_15280581483b47f18aa57296f25d8334",
            "value": "Downloading: 100%"
          }
        },
        "384f16c5e1d54c2397d9c75d06c1bfbd": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "389af45e82694958b597e0c25529f6c0": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "38e443a3d6124361b5fc1acaaeaeb93e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "38efabdd23f74921951c71ae079405d7": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_80d43815d1fb49afba8f36b0cd86a762",
            "placeholder": "​",
            "style": "IPY_MODEL_c26834a8a1474c839203335c9c57237d",
            "value": " 29.5M/? [00:00&lt;00:00, 62.6MB/s]"
          }
        },
        "39722132ac22429fb0d99e5545566715": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3a7a4945d0f04ce5ac54621bf1d02698": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3ac1b5e8c3764a85b55d2df5a5890eed": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "3b037f3257684895ba25778397aeee19": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": "20px"
          }
        },
        "3bfa7d3b037141758002c4b222db6aa8": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3ce1683b100b4e249a4272220b0b6bed": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "3dee85fadf5648c6b8b109ee428783fa": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "3e24127c11c243e9aa7c300d4950f2d1": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3e842b6e81c041419f630f7bbf69e0d2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3ef01336fc77403789e6c7bf2fe7fbd4": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3f549e3d727043358247df4eddcc7c07": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "4165d6cf31964bc9bad2e555153a6c94": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8d3dc00e3e774bd99e52e494f0a9818a",
            "max": 352833716,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_b1bed1525f1b4abebc507eb9f3ae1ba7",
            "value": 352833716
          }
        },
        "42b2c0fc94964d76b2c0d606bf26c891": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_59864ed939c64473ad1da8ed9568304b",
            "placeholder": "​",
            "style": "IPY_MODEL_9d734fd174764c0492ca1b7593edced2",
            "value": "Downloading: 100%"
          }
        },
        "43f0edc85a3e447f92b22820c71a7b01": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_93f474326a6e41809bdf32621655423e",
            "max": 7600,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_edd8accd2cd444ac80e7d7dd9e93c605",
            "value": 7600
          }
        },
        "43f85ac6c8ae4044a0e4443d7da79eb3": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f3e74d9727564ed3ac609ebb3ec74737",
            "max": 698,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_16ebe063385841faba7f79ae57dad474",
            "value": 698
          }
        },
        "4429b3f77f2941468d72ba1b17821d9a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "44408e7021a34ed7b5aeeed803f48f30": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_7ffad741bcf84ea59f3a41239cd92045",
            "placeholder": "​",
            "style": "IPY_MODEL_38e443a3d6124361b5fc1acaaeaeb93e",
            "value": ""
          }
        },
        "45ba9ea28ab942a0a46bb51abc2d51d6": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "466c9e5977b7463b84a6a4d91458bc6d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "4772a52c663241ba9b5314d887fd47de": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "4826faba9b6049929733e16542881e14": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "49b9d7f7b9e645ed8273bc1fbbd1bbf5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_87b0156a2a4a4f27bb6c2232c29b8723",
              "IPY_MODEL_69b9496a0d6d44fdaa43b7e5a607c407",
              "IPY_MODEL_eea82cd791c141f799b256e9023eb0c2"
            ],
            "layout": "IPY_MODEL_9da3c4afad1b441386ad7f10b32d5db9"
          }
        },
        "49d1ba4b752145f4ac0668dfdb4b40b0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "4a41a1e705b549f8b094e5a1432ea389": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "4b00c04c8bf34288b10505c897f6516f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "4b356a43e7784e1e86b250a0aa5e4f8c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "4bde8be94bb945b7869e4fa261d686dd": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "4c2f7522315c4606b6e14ea7a7d1a607": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_091851ddeeb6437aa1a8cc8fbe2bc17c",
              "IPY_MODEL_8b0669653acf44bc877c194a5e57944a",
              "IPY_MODEL_33d93bd8482841f5ad2bc99317c85c27"
            ],
            "layout": "IPY_MODEL_74e6b1af5e5942578decd8a2d806f5e7"
          }
        },
        "4dce807720b442788b75eecf25174e86": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "4f35888d5ef74153b2c1bf739cce1f9c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "4ffa8b39a34a4433bcce70f713bd7f2a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3a7a4945d0f04ce5ac54621bf1d02698",
            "placeholder": "​",
            "style": "IPY_MODEL_af69668241644ef4b5cb06223a565d9f",
            "value": " 232k/232k [00:00&lt;00:00, 249kB/s]"
          }
        },
        "5013660e1a0149e1b228eb15ed544e5c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "50549cabedf24e11b3bb689bb32b4e47": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_42b2c0fc94964d76b2c0d606bf26c891",
              "IPY_MODEL_5faeb1b875c341f5863f170005a6858c",
              "IPY_MODEL_91665c2c2aa64904b5ca2b50c82470bb"
            ],
            "layout": "IPY_MODEL_e84414f3f9a2410c8983e332bed42526"
          }
        },
        "5096a37007de48ab8a6f3dd6b3a9e68f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5135f2da12774e49b3941a75ee539fb0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_9f63d34305c544cda90ae4a9ee60449b",
            "max": 500,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_f4318ba53a2b49ce95af89537538d815",
            "value": 500
          }
        },
        "5199e127b316467d872958d3d03fb8c5": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "521278900d8241088363e811991c1e66": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "521aff2ec667482496021dda759f58d3": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "527265f507a946a283f7437e679fa909": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "52a04a50c3bb491dabd849005adfe60b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_ddd780b848994a3eaea4257b5b3f6dee",
            "placeholder": "​",
            "style": "IPY_MODEL_85fa69e6cb8c4b84b8bcb7c36bf2d8dd",
            "value": "Downloading: 100%"
          }
        },
        "52ce268c3a4d4c1da647dda194d2e511": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "55783e273f894123b18c1ab84fe996a1": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f1e394f5978f44de9d7f539e78699582",
            "placeholder": "​",
            "style": "IPY_MODEL_d1f5bcde8d754af18fdd052c98a6d81b",
            "value": " 29.0/29.0 [00:00&lt;00:00, 688B/s]"
          }
        },
        "55ddf3f2dca4458ca9866c4600aa1186": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "55ed66dd456e4f19ae98ce739da552b8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_c59172344f204280a756b106f69ed793",
            "max": 28,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_e35b435f72d4403e9ac00852fad7eef7",
            "value": 28
          }
        },
        "5661f01f86c849568a78e21c7fce7f5b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "57c21a5483ec40a998a7ef66301a41ae": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "583320fb40344b9bbb1887848173ca45": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "586aa53e10754cb8948e3636da7aef9c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "588dfe8a747a4cd2a9ca35bba8898808": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "58c2c3c4502940a38c6f842850f43b08": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "59864ed939c64473ad1da8ed9568304b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "59add760ed794047bbb548f5c09e64ac": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_ff32978b71364a37a6724a5ee508bed3",
              "IPY_MODEL_43f85ac6c8ae4044a0e4443d7da79eb3",
              "IPY_MODEL_1e9772f564fa4d4390518664a1595197"
            ],
            "layout": "IPY_MODEL_aedd4dc023644d5cb76f33462ae965c7"
          }
        },
        "5a37cba6a0da4612bbcc177d0d547d31": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_671bd251ea704b32894d76b15c824772",
            "max": 120000,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_9a019409c8154d869f71a5a8ebcc4ddf",
            "value": 120000
          }
        },
        "5a4c991a4e2247e6818707680d47a560": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5b85631ac5a446e5884020fb7ba4b96f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5b952890c4ac48ed8fd729ed1322a6b8": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5ba11688b1a34d1298a3c0b164b7b3cd": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": "hidden",
            "width": null
          }
        },
        "5bcd549955304dbeba4003e13518ab74": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5c1987d5e4824dda8818f2d2187a0666": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5ca4915e26b34b1d8d125f63a9b3f8ff": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_85ae16107d9443c2bbced1a4df0ca71b",
            "max": 500,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_fda6478d993342a798d281d6da9d8cd0",
            "value": 500
          }
        },
        "5cb53426cc29476dadb4c2fb88c5b5ad": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5cef548a19104d2a910071036dccc588": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "5d32f0e96b11429fbd8f2d7aaa3cabe8": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "5e5b7d74bc1545bcbf3f2cc61edb660b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_28aaceda6c194ab09a986b360e9f1d43",
            "max": 492,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_f4d7b4451ac64e9e9cba9c79a2a5ed58",
            "value": 492
          }
        },
        "5f1eef839eb24e25bbf667d0c23d2378": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_7ff7ecf2e5f44961becb04faa0a3048b",
            "max": 570,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_cad503bc0c3346e2abf5b25d355305d7",
            "value": 570
          }
        },
        "5faeb1b875c341f5863f170005a6858c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_d1da106abef7438891a93504e436140c",
            "max": 802397,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_fa11c85885b34c0c8e105c3cec6fbb83",
            "value": 802397
          }
        },
        "5fd3ce3c4f1e4751b6e1567b1d0cc447": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_c5bcb4f5e0b740f6adaaecbc9b38a8e0",
              "IPY_MODEL_36a31d0fa707478285f094582fb2b090",
              "IPY_MODEL_8b0b3d0a39df422ab85e98147252c37d"
            ],
            "layout": "IPY_MODEL_20d9a77f209d473bb1d2e751d706913e"
          }
        },
        "5fe53cfebe1f49fa9b16d3f6183ef62c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_a9a3dd627afd4247b8f67d9775489c37",
              "IPY_MODEL_a1818b02b2c44451a27295e72d3147b6",
              "IPY_MODEL_249fb46f6f33468d8a40e76a76371e63"
            ],
            "layout": "IPY_MODEL_2dab7eeda51541a98b42f291541dde89"
          }
        },
        "601c8305985b422b84ef3ad5cf7561e9": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "60e62383e2ec45deaf196e18b02f39af": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_981a2d85af5849b4802f2c32489c9100",
            "placeholder": "​",
            "style": "IPY_MODEL_30df52fae0f74e1caa14dab8f4689aff",
            "value": "#1: 100%"
          }
        },
        "62930768884441de957525daceb027de": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "62ca2598d0ff440483940669dde3f193": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_fd921171897a4ffc99562d675733b308",
            "max": 380,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_3dee85fadf5648c6b8b109ee428783fa",
            "value": 380
          }
        },
        "62f64323983d426c8762984fd25bb02e": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "636cfa48a2c647efa9ada29b42c37a35": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_57c21a5483ec40a998a7ef66301a41ae",
            "placeholder": "​",
            "style": "IPY_MODEL_4dce807720b442788b75eecf25174e86",
            "value": "Downloading: 100%"
          }
        },
        "63a8257eab9d4099a9611b13cdf8b20e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_739a02fcd76f4b7ab68f4044d38fbad5",
              "IPY_MODEL_f101c0bc1fc34bbdbe6b386d7858b593",
              "IPY_MODEL_fb68c9499ecb4403939127cd750718eb"
            ],
            "layout": "IPY_MODEL_5661f01f86c849568a78e21c7fce7f5b"
          }
        },
        "64413aa525174ffc90e7bed22b532b7b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "645f9c5a7dc84530a409a75b0373c283": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "64f9c055090a4861b33f8ed15a858daf": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "65fd5c5e9b2c4cbdb4fe0c3314ae29bb": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5c1987d5e4824dda8818f2d2187a0666",
            "placeholder": "​",
            "style": "IPY_MODEL_340c96ed160c42b191291545dfe6848c",
            "value": "Downloading: 100%"
          }
        },
        "66b4340421ba4bb88797ed6ebc04c177": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "6709c3d560714b06b77382f279440ef6": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "670ce1a5a4a3477e970db3210192ec4a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "671bd251ea704b32894d76b15c824772": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "674a1753257541edb0be46d7811c6e72": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "67ce1f7d87c34f4dafdcd908c7dd7b84": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f8bb3d12d0144d72b7f5bed4055d2b29",
            "placeholder": "​",
            "style": "IPY_MODEL_dc775613d7d74f4a84f0731f595eaba2",
            "value": "Downloading: 100%"
          }
        },
        "67e91a4c9376417dbb9916e83f5ccdd4": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "67eb4a8763f648c1a4934e076c64bb2b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "6835c929cd844a7f882765f68bd8a7e2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "689de7097be44956ac7d9a5736b8e5bb": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "693c8df85dde460b87f0624d33e196ed": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3e24127c11c243e9aa7c300d4950f2d1",
            "max": 4063,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_6f98a870b3684a79977334c15c9d1324",
            "value": 4063
          }
        },
        "69b9496a0d6d44fdaa43b7e5a607c407": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_7656cd9f368f4469a4c7f410e5977ec2",
            "max": 231508,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_6c9e1f3b770b429bb6ee5ef7234dd78f",
            "value": 231508
          }
        },
        "6bd38346f1fd4246b5f8f6aad5176773": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "6c2c388a465f447a976d3427b59bad7a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_09e77e9a34f24fedbdb90b2112f27347",
            "placeholder": "​",
            "style": "IPY_MODEL_5cef548a19104d2a910071036dccc588",
            "value": "Downloading: 100%"
          }
        },
        "6c6139aefafc430b9ee6ce8644399eda": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5096a37007de48ab8a6f3dd6b3a9e68f",
            "max": 437983985,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_4b356a43e7784e1e86b250a0aa5e4f8c",
            "value": 437983985
          }
        },
        "6c9e1f3b770b429bb6ee5ef7234dd78f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "6d3fa316dfa24d20bda288ddfb71b4c8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "6f98a870b3684a79977334c15c9d1324": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "707373ef81ca460d88b754c71800b72c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "70ae5680a40a4e4fb7f71c6f951f1479": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "713c08d937ca4e31a8b8f6942a487b7a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_a4424cb16ec849ee96df431f52a57e29",
              "IPY_MODEL_5f1eef839eb24e25bbf667d0c23d2378",
              "IPY_MODEL_bad437a15b834e3e8ca6ca67b4860b7d"
            ],
            "layout": "IPY_MODEL_836bd677b7b040e2812fa3d3cf39b491"
          }
        },
        "718e86634d1343559659a545f36abb42": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "71a9ceb690724a8ab688acbf67c68113": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "71c9f44ab15048879ce62315ff0c48fb": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_ee1d498e7e20480988712721acd42b12",
              "IPY_MODEL_5e5b7d74bc1545bcbf3f2cc61edb660b",
              "IPY_MODEL_18033c64e10e4f31b663e47d70bc8fe2"
            ],
            "layout": "IPY_MODEL_384f16c5e1d54c2397d9c75d06c1bfbd"
          }
        },
        "72691438bd3443e496380c56af496eaa": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_44408e7021a34ed7b5aeeed803f48f30",
              "IPY_MODEL_e54ba4ac5c6949d7bcb97eb9729d6ada",
              "IPY_MODEL_335a51f7dd5548c3947a6288495c1737"
            ],
            "layout": "IPY_MODEL_2bd6ec97d073442eb08d141c07fd199d"
          }
        },
        "735a419b8c944ac9806a9c009dc6a09f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "737eb4f444f5460eadbedf755e488804": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "739a02fcd76f4b7ab68f4044d38fbad5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5bcd549955304dbeba4003e13518ab74",
            "placeholder": "​",
            "style": "IPY_MODEL_9a83be7115ca4b6dbb90715c551da421",
            "value": "100%"
          }
        },
        "7431585ef62e4e26a39006d3fd8aec8c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "74acdd9bc09c438ab1f48c8c762c4a07": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "74d518f82ecb4021ade1a713de64104b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_583320fb40344b9bbb1887848173ca45",
            "max": 762,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_9b421d1eb87a4d9fbde251053fcf6c2c",
            "value": 762
          }
        },
        "74e6b1af5e5942578decd8a2d806f5e7": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "75efbc49c6e84d9f8146c04de0f4daa8": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "7656cd9f368f4469a4c7f410e5977ec2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "76932440413a4331b2f4754b7b226963": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "7703ade855614a7099230146866d17e6": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "785bfc66d62b4e109573f605b5fec286": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "7b964fbfd635438f96ff1858ca33c0e5": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "7b9c932216c24b61b1f8c9d3fc315863": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5b952890c4ac48ed8fd729ed1322a6b8",
            "placeholder": "​",
            "style": "IPY_MODEL_37643f4536624f2b93c5ef1028de495d",
            "value": "Downloading: 100%"
          }
        },
        "7ce42569074943d3af0ca12b2ac351d7": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "7da43125ff734d4580f56def5c2a24a6": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_973c58059452472697894eab30adea3b",
            "placeholder": "​",
            "style": "IPY_MODEL_cdc952769a3841de91d4d49513b5f4c8",
            "value": " 1.86M/? [00:00&lt;00:00, 21.8MB/s]"
          }
        },
        "7e7492861d2d4334aaa522829fe52d0a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_135d819a8d7f4e3ba85bed56b318a0d3",
            "placeholder": "​",
            "style": "IPY_MODEL_601c8305985b422b84ef3ad5cf7561e9",
            "value": "Generating train split:  99%"
          }
        },
        "7fcf5fa6a46849968f92fd00b2670543": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "7fdf1d094b4e4693acc53477fd2379c2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "7ff7ecf2e5f44961becb04faa0a3048b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "7ffad741bcf84ea59f3a41239cd92045": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "80d43815d1fb49afba8f36b0cd86a762": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "80f274ee73da4a2dbfb3d56d48636ef5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "8111135a8f6f4c6b8e634fe978045a56": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "81bd92c195424af2b552038bb2240473": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "836bd677b7b040e2812fa3d3cf39b491": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "847410262bdd45919fdb4dbb18a5df2c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_3828d19ebc43415f8ff87568418ec9b2",
              "IPY_MODEL_f79a56e0fe024b9682ec79d134201412",
              "IPY_MODEL_a5e970c0cf8a48ae8d2cb45058887d6b"
            ],
            "layout": "IPY_MODEL_aab826eacf3b429a8b67bb4c42698154"
          }
        },
        "84ccee50a10d47f392d094a53c5223d9": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "85ae16107d9443c2bbced1a4df0ca71b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "85fa69e6cb8c4b84b8bcb7c36bf2d8dd": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "864803498d9d485b854f6dc22bf8e7f7": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_e276dfc72b4f4b42a5a93241c7dcd0b4",
              "IPY_MODEL_30caa70b8e2b4ca28af34fb14cd0cf86",
              "IPY_MODEL_233c55aac32242f6be5085631989f7c8"
            ],
            "layout": "IPY_MODEL_c1d0a6be0e0b402696f8f759ecf692e2"
          }
        },
        "868d87d61432416fb23969b8405cc628": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "87349896035b48a99c10939bc954222b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "87b0156a2a4a4f27bb6c2232c29b8723": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_707373ef81ca460d88b754c71800b72c",
            "placeholder": "​",
            "style": "IPY_MODEL_068453e620e34f0cb09aa4cd2a9043c6",
            "value": "Downloading: 100%"
          }
        },
        "882871406b0043229ea41e2bc7e20cd9": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_28ac8463e3f14e899b7e16553d991d41",
            "placeholder": "​",
            "style": "IPY_MODEL_71a9ceb690724a8ab688acbf67c68113",
            "value": " 570/570 [00:00&lt;00:00, 17.6kB/s]"
          }
        },
        "88b39f446a4d46f5a5748d8bdf4311cf": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "88f0a8f7296e47af8f751190bc11b065": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8975b0dad9b34d4bb3d654e833af9915": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f832dc2bf7e048ccb168cc96bc8fb8f2",
            "placeholder": "​",
            "style": "IPY_MODEL_a94420be4e1d4aa49774c2fce210aeaa",
            "value": " 500/500 [00:01&lt;00:00, 321.35ba/s]"
          }
        },
        "89c56c8131c74168ad32930ab11500f1": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "89d19eb4b1a243bf8318b037f3eb1eb2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8a2dcb7fe90040328cacd70146c48202": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8b0669653acf44bc877c194a5e57944a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_35d7392e1f7441269689a5260050bf8d",
            "max": 2653,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_21e3b941f0b24664895ad9913206d11c",
            "value": 2653
          }
        },
        "8b0b3d0a39df422ab85e98147252c37d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_81bd92c195424af2b552038bb2240473",
            "placeholder": "​",
            "style": "IPY_MODEL_fee18278d97f4191a086db8dc185409b",
            "value": " 436k/436k [00:00&lt;00:00, 300kB/s]"
          }
        },
        "8b54d237a7cc43beaeb6a1edee13714e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_bcd5a9c91e52470c9c285fda8a659a94",
            "placeholder": "​",
            "style": "IPY_MODEL_fa07548194e244b8b655381e073abb9d",
            "value": " 762/762 [00:00&lt;00:00, 24.4kB/s]"
          }
        },
        "8befeadf18114bdba529e65426de91e2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8c15f48271214e97b30e3f5ad1658f72": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_9618539b7cdb432d8e374aa593ad7385",
            "placeholder": "​",
            "style": "IPY_MODEL_d1b47ba0a7e84c3989a232e2e8532f9c",
            "value": " 438M/438M [00:18&lt;00:00, 31.9MB/s]"
          }
        },
        "8c7334d274ee4d3c9d58a0cc39838984": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8cbd54818cbf49e7abe5a9b404e30320": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8d2ad2b28f1644f09ff21d95df9bf0c0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "8d3dc00e3e774bd99e52e494f0a9818a": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8d664b21ce974899bb6a7aaaea21a69e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_a8650804aeaf4a37b9732b890791d06d",
            "placeholder": "​",
            "style": "IPY_MODEL_d3f574de1ff448178358f3a4ddf803c9",
            "value": "Downloading: 100%"
          }
        },
        "8d85f1f723d340d19fb50105344bab69": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8ee9eb7d0c8c4f2998dc41835994e442": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "8f1bf0c58d1e493a8a00e96ce09590da": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8f96e5e63c61463181ad8daed8f104e0": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "8fc5b50cd6b045a7a131a94d44a67c4a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8d85f1f723d340d19fb50105344bab69",
            "placeholder": "​",
            "style": "IPY_MODEL_70ae5680a40a4e4fb7f71c6f951f1479",
            "value": " 778k/778k [00:01&lt;00:00, 782kB/s]"
          }
        },
        "8fd8aa6532e24643b264157c63463471": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "90beda2982944894a9450e5a509daf2e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_60e62383e2ec45deaf196e18b02f39af",
              "IPY_MODEL_fd2c6d3ac0754db3acad1e88ade2c86d",
              "IPY_MODEL_c5f0c1a82e1f422983431c3463ef9604"
            ],
            "layout": "IPY_MODEL_66b4340421ba4bb88797ed6ebc04c177"
          }
        },
        "91665c2c2aa64904b5ca2b50c82470bb": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_c2ca862568404b5c82a6170dbd90c437",
            "placeholder": "​",
            "style": "IPY_MODEL_521278900d8241088363e811991c1e66",
            "value": " 802k/802k [00:01&lt;00:00, 615kB/s]"
          }
        },
        "91759f59f8604393a0d72c623b1b97af": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8befeadf18114bdba529e65426de91e2",
            "placeholder": "​",
            "style": "IPY_MODEL_8ee9eb7d0c8c4f2998dc41835994e442",
            "value": "Downloading: 100%"
          }
        },
        "91e3ec446c5d4030853dca426e304f9c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "92df20c724504084af9595139f251978": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "93f474326a6e41809bdf32621655423e": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "94ac791468ea4909a84c353d5bc7611d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "94d4edee3f7a406281f694241eb143ab": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9618539b7cdb432d8e374aa593ad7385": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "96906f2b2dcf4a34bcb6418d44751575": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "97195ac813af429084132711525e4a91": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_37731c0ff1954df5893b00dcb7f0b6d9",
            "max": 778395,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_c6c2e069da7a41e583bc0834a54a9b78",
            "value": 778395
          }
        },
        "971ee1ec3e25467fa861cf30207f25a7": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "973c58059452472697894eab30adea3b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9780793b16aa4029ae550481db873a7d": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "978de6521f3d4b8b9ac305b9db7a0ff1": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "981a2d85af5849b4802f2c32489c9100": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "99233039049945f8b9db2d9f13e084e6": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9964fe9b3e0f464ba4de95aeefc75422": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "999d7a37dd8843b1aa9920c0da638c81": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9a019409c8154d869f71a5a8ebcc4ddf": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "9a83be7115ca4b6dbb90715c551da421": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "9b39591d155d4d5b9bf7b9c37c928332": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_52a04a50c3bb491dabd849005adfe60b",
              "IPY_MODEL_1d9d2cf9cd59425497adf32e4dd59261",
              "IPY_MODEL_882871406b0043229ea41e2bc7e20cd9"
            ],
            "layout": "IPY_MODEL_2af41c4c23e24e3db73a9003a32d0097"
          }
        },
        "9b421d1eb87a4d9fbde251053fcf6c2c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "9b601f47611b4281b2b2d1fcab39c38b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9b6c07e98b4c4e95bdddc7dac23d69cf": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "9ba8816c2f51491a8596bb33fc8998d8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "9ca3d1acbf904aa2b7f9f1f1475a5e9f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9ca45bbcbdbe4eab98cb67ed1249d8fb": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "9cc17f6d1a40454b8db2052160168a12": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_ec288e953b244616befab8429457c77b",
              "IPY_MODEL_c6b3661a3447492ab9e5b8d1b3aae05e",
              "IPY_MODEL_3586240e502f4255ab0d3d66e79b449e"
            ],
            "layout": "IPY_MODEL_64f9c055090a4861b33f8ed15a858daf"
          }
        },
        "9d734fd174764c0492ca1b7593edced2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "9da3c4afad1b441386ad7f10b32d5db9": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "9ef088f4f6d244738be3a42da4396635": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "9f63d34305c544cda90ae4a9ee60449b": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "a0f74a508afc44ed9c618e1d181807a5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_2cb7d45a1a804c109e67e9a6c50976ee",
            "placeholder": "​",
            "style": "IPY_MODEL_2ba33d1e19ac48149fc2c78e1baba2a9",
            "value": " 456k/456k [00:01&lt;00:00, 476kB/s]"
          }
        },
        "a10e589c393744f58fc09cc1b97c8df5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "a1818b02b2c44451a27295e72d3147b6": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_e54c1368445541539af7f188bc6196cb",
            "max": 42,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_521aff2ec667482496021dda759f58d3",
            "value": 42
          }
        },
        "a1cb51aa9a3246eda0676252be68909c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "a1d20c56010243e38917d595f6514253": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_097505d47ab44732968670d7971503e4",
              "IPY_MODEL_b0de99cc03ce43bdba8108442f8db6f3",
              "IPY_MODEL_f272c61ecafb44299bcc8974f6062dba"
            ],
            "layout": "IPY_MODEL_1a9aca81153942569a0dbccb93555619"
          }
        },
        "a303620e1ed142658667b32cee4b7e70": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "a41948d2db514c168bb2801bb7c539d8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8c7334d274ee4d3c9d58a0cc39838984",
            "max": 1339166,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_ca99d452a20643fd996ff0b9064b3e1f",
            "value": 1339166
          }
        },
        "a4424cb16ec849ee96df431f52a57e29": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3e842b6e81c041419f630f7bbf69e0d2",
            "placeholder": "​",
            "style": "IPY_MODEL_b81956f3a32f4d8cb6dbe07a62867627",
            "value": "Downloading: 100%"
          }
        },
        "a45abec670d242799e43d5badb75fa99": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "a4befd44d0b5402e8bcb61f64162ff43": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_380c55c2680a4f85a81cfb266eace834",
            "placeholder": "​",
            "style": "IPY_MODEL_94ac791468ea4909a84c353d5bc7611d",
            "value": " 28.0/28.0 [00:00&lt;00:00, 884B/s]"
          }
        },
        "a5e970c0cf8a48ae8d2cb45058887d6b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_d8afe82dd0454aeeb34099fd21ccf726",
            "placeholder": "​",
            "style": "IPY_MODEL_04a785e5695545f8905306feee92b618",
            "value": " 2.59k/2.59k [00:00&lt;00:00, 84.0kB/s]"
          }
        },
        "a642a8054e0a4ba08a516b18498ee769": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "a7bf4c1831c54aa298d2cd3450ed66fa": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "a8650804aeaf4a37b9732b890791d06d": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "a94420be4e1d4aa49774c2fce210aeaa": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "a9a3dd627afd4247b8f67d9775489c37": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_335b62d41c464d3a8cd202471a0154d7",
            "placeholder": "​",
            "style": "IPY_MODEL_aac253fca0774d0db2c8f27ca6f24bca",
            "value": "Downloading: 100%"
          }
        },
        "aab826eacf3b429a8b67bb4c42698154": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "aac253fca0774d0db2c8f27ca6f24bca": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "ab16e2668a6647dab78c2669e945917d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_58c2c3c4502940a38c6f842850f43b08",
            "placeholder": "​",
            "style": "IPY_MODEL_23044cf6a4d04064b4cb3dbf46e22041",
            "value": "Downloading: 100%"
          }
        },
        "ac0350f2087a4d74b3dee0de2400545f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_378dd87d8d1e499880a8ce6e6ff5f6ef",
              "IPY_MODEL_e799ed767acd4c5f8825bb1bbf7f7bda",
              "IPY_MODEL_feed7ab81f894f04b4e485f54ad25ff0"
            ],
            "layout": "IPY_MODEL_0ff7be06dab04b83b1bbc27f67f1bae6"
          }
        },
        "ac6217065b744df9a9ee700a3f94f655": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "ac6aaa3c18cd4824971c422068858a53": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "acfe14fcd0fc484b957fe97cfcb34c4c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "ad3776cc4d204d2daaa0bc18c24bdcba": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_88f0a8f7296e47af8f751190bc11b065",
            "max": 11045148,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_ec57f9449406445a9e91e35c0236da0b",
            "value": 11045148
          }
        },
        "adf45441dafd499bb5eab3b3a969a3c0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "aedd4dc023644d5cb76f33462ae965c7": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "aeefe750041e456c8294fa00d2866cdb": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_ab16e2668a6647dab78c2669e945917d",
              "IPY_MODEL_1480d86a28d842dbaec31ca26b3486fc",
              "IPY_MODEL_b6e133ea752b46b987eadac7d996108f"
            ],
            "layout": "IPY_MODEL_55ddf3f2dca4458ca9866c4600aa1186"
          }
        },
        "af69668241644ef4b5cb06223a565d9f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "b0de99cc03ce43bdba8108442f8db6f3": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f62aa191dfa443678339b194a59d75ff",
            "max": 466062,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_f20fabbf6f064b8a8be0c99065f707b4",
            "value": 466062
          }
        },
        "b173b9397ed748b98aa18e9b4b739fca": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_b611fcfb99e84693954b41743fbf321a",
            "max": 231508,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_8d2ad2b28f1644f09ff21d95df9bf0c0",
            "value": 231508
          }
        },
        "b1a60018182448fc9ca922efed6a4ce4": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "b1bed1525f1b4abebc507eb9f3ae1ba7": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "b2966a02f9b74b7895c525b6fb21597d": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "b2990a41d73b4f058a1cf336e78b9aea": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "b29bd3a6f57d414ea8c2625317daf851": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_2da10880d1da4b7eba255333d171d653",
              "IPY_MODEL_74d518f82ecb4021ade1a713de64104b",
              "IPY_MODEL_8b54d237a7cc43beaeb6a1edee13714e"
            ],
            "layout": "IPY_MODEL_148fd6dd13ca40199c14bc396a741d83"
          }
        },
        "b2be2c8915c842a3a197c00b5ceeb77f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "b4ca9d8896d6480093c780f9c0f4d3ee": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_ed3abc2e209f4590a4626efa3d2d24be",
              "IPY_MODEL_693c8df85dde460b87f0624d33e196ed",
              "IPY_MODEL_ef5ff713eb5c41738bdd73921418308c"
            ],
            "layout": "IPY_MODEL_b2be2c8915c842a3a197c00b5ceeb77f"
          }
        },
        "b4d53b1db23e46278f2a68258594c9da": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "b4ef3d467ecb4d63a3a8e90b9db13fb3": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "b611fcfb99e84693954b41743fbf321a": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "b62ce40a257541c496cb5c8c914f670e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_df3806b3fc674e4f9f899d5dbb6a4115",
              "IPY_MODEL_e98979e4e72441019858ec61946f934f",
              "IPY_MODEL_7da43125ff734d4580f56def5c2a24a6"
            ],
            "layout": "IPY_MODEL_92df20c724504084af9595139f251978"
          }
        },
        "b6e133ea752b46b987eadac7d996108f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5a4c991a4e2247e6818707680d47a560",
            "placeholder": "​",
            "style": "IPY_MODEL_76932440413a4331b2f4754b7b226963",
            "value": " 301M/301M [00:05&lt;00:00, 57.3MB/s]"
          }
        },
        "b701e33b12a14bac98bab4d888e72c43": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5d32f0e96b11429fbd8f2d7aaa3cabe8",
            "placeholder": "​",
            "style": "IPY_MODEL_737eb4f444f5460eadbedf755e488804",
            "value": "Generating test split: 100%"
          }
        },
        "b81956f3a32f4d8cb6dbe07a62867627": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "b884187aaa5844bdad69287202c10509": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "b8d4aa7e1c6a4e4898f3fba303e318bd": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_b701e33b12a14bac98bab4d888e72c43",
              "IPY_MODEL_43f0edc85a3e447f92b22820c71a7b01",
              "IPY_MODEL_e231320115e544239d9a800d9cd32094"
            ],
            "layout": "IPY_MODEL_5ba11688b1a34d1298a3c0b164b7b3cd"
          }
        },
        "b9474845c2d24c6f91420bb23ca0e149": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_88b39f446a4d46f5a5748d8bdf4311cf",
            "max": 466062,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_7fdf1d094b4e4693acc53477fd2379c2",
            "value": 466062
          }
        },
        "ba4f66d47deb45e28d559085f3eff4ae": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_67ce1f7d87c34f4dafdcd908c7dd7b84",
              "IPY_MODEL_6c6139aefafc430b9ee6ce8644399eda",
              "IPY_MODEL_8c15f48271214e97b30e3f5ad1658f72"
            ],
            "layout": "IPY_MODEL_64413aa525174ffc90e7bed22b532b7b"
          }
        },
        "bad437a15b834e3e8ca6ca67b4860b7d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_4772a52c663241ba9b5314d887fd47de",
            "placeholder": "​",
            "style": "IPY_MODEL_1c28b4c4a1bd44ddbed1028e3a755bb5",
            "value": " 570/570 [00:00&lt;00:00, 15.3kB/s]"
          }
        },
        "bafda22907ec4b6e929dc553f936e18d": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "bb474e5c83f847078224674ef58ab55d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_7b9c932216c24b61b1f8c9d3fc315863",
              "IPY_MODEL_97195ac813af429084132711525e4a91",
              "IPY_MODEL_8fc5b50cd6b045a7a131a94d44a67c4a"
            ],
            "layout": "IPY_MODEL_ac6217065b744df9a9ee700a3f94f655"
          }
        },
        "bc150147b2c845f3aead1c4b1fa09c61": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_978de6521f3d4b8b9ac305b9db7a0ff1",
            "max": 456318,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_bfa2bb61bc534200bdb61bc28bc09b53",
            "value": 456318
          }
        },
        "bc4291a502d54678bfe7fde6b1c51d33": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_36e0dc35ec09429b8c61529dcbf88cae",
              "IPY_MODEL_a41948d2db514c168bb2801bb7c539d8",
              "IPY_MODEL_cd66da55631c42b3a75eb44b72b06ac2"
            ],
            "layout": "IPY_MODEL_00a90b713cd14806b103195a3d71b091"
          }
        },
        "bc51d76c3a854586b7b4832771a112bf": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "bcd5a9c91e52470c9c285fda8a659a94": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "bd052d320db8492c9b3aa1cf29e4a9cf": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_74acdd9bc09c438ab1f48c8c762c4a07",
            "placeholder": "​",
            "style": "IPY_MODEL_f8eaa261c4df44cea66453f6233555f9",
            "value": "Downloading: 100%"
          }
        },
        "bee129673ec34431932b80148b50aab0": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "bfa2bb61bc534200bdb61bc28bc09b53": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "c110d11390cd4e9ca915af693a681c4a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_fe48ae258e904d6aaac09c728868ce41",
            "placeholder": "​",
            "style": "IPY_MODEL_adf45441dafd499bb5eab3b3a969a3c0",
            "value": "Downloading: 100%"
          }
        },
        "c11372c27bb34a03a26971a77791141a": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_65fd5c5e9b2c4cbdb4fe0c3314ae29bb",
              "IPY_MODEL_cefc16b8f9c94fc0aee7ba72a1c4aeb0",
              "IPY_MODEL_d35f83b7433e4f1bb857789c9738bf21"
            ],
            "layout": "IPY_MODEL_999d7a37dd8843b1aa9920c0da638c81"
          }
        },
        "c1d0a6be0e0b402696f8f759ecf692e2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "c26834a8a1474c839203335c9c57237d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "c278bbf65b6d43c0b82af9a3cab5ef9e": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "c2ca862568404b5c82a6170dbd90c437": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "c2de5f63d1654c869877718fad50265c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_de5cb07d322c4bb4ac96700939e98dae",
            "placeholder": "​",
            "style": "IPY_MODEL_ee2cbeb2ea6749658bfb764f181c115f",
            "value": " 353M/353M [00:09&lt;00:00, 54.0MB/s]"
          }
        },
        "c59172344f204280a756b106f69ed793": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "c5bcb4f5e0b740f6adaaecbc9b38a8e0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8a2dcb7fe90040328cacd70146c48202",
            "placeholder": "​",
            "style": "IPY_MODEL_b4ef3d467ecb4d63a3a8e90b9db13fb3",
            "value": "Downloading: 100%"
          }
        },
        "c5f0c1a82e1f422983431c3463ef9604": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_bafda22907ec4b6e929dc553f936e18d",
            "placeholder": "​",
            "style": "IPY_MODEL_785bfc66d62b4e109573f605b5fec286",
            "value": " 380/380 [00:01&lt;00:00, 344.41ba/s]"
          }
        },
        "c6b3661a3447492ab9e5b8d1b3aae05e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_f9ac22e242984baeaae8ddae3748003c",
            "max": 1042301,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_2de4643e661d448caaa7a1a7e660c6b0",
            "value": 1042301
          }
        },
        "c6c2e069da7a41e583bc0834a54a9b78": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "c923a57cac3a44389ad8b8bf0920a8aa": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "ca79683bd37243f98b5f10f2e6f373c4": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "ca99d452a20643fd996ff0b9064b3e1f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "cad503bc0c3346e2abf5b25d355305d7": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "cd66da55631c42b3a75eb44b72b06ac2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_7fcf5fa6a46849968f92fd00b2670543",
            "placeholder": "​",
            "style": "IPY_MODEL_ea5c3ecdef4f46509e6dc204abee8184",
            "value": " 1.34M/1.34M [00:01&lt;00:00, 821kB/s]"
          }
        },
        "cdb2e3a16db04678a4a6d5d23931dacc": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_6c2c388a465f447a976d3427b59bad7a",
              "IPY_MODEL_b173b9397ed748b98aa18e9b4b739fca",
              "IPY_MODEL_4ffa8b39a34a4433bcce70f713bd7f2a"
            ],
            "layout": "IPY_MODEL_62f64323983d426c8762984fd25bb02e"
          }
        },
        "cdc952769a3841de91d4d49513b5f4c8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "cefc16b8f9c94fc0aee7ba72a1c4aeb0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8f1bf0c58d1e493a8a00e96ce09590da",
            "max": 2503,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_a45abec670d242799e43d5badb75fa99",
            "value": 2503
          }
        },
        "cfec14d18e554e998de7ce8cc71496f8": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_21332697cbd5478e9a2158302f71ed84",
              "IPY_MODEL_4165d6cf31964bc9bad2e555153a6c94",
              "IPY_MODEL_c2de5f63d1654c869877718fad50265c"
            ],
            "layout": "IPY_MODEL_5013660e1a0149e1b228eb15ed544e5c"
          }
        },
        "d1b47ba0a7e84c3989a232e2e8532f9c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "d1da106abef7438891a93504e436140c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "d1e2d720723b4d99bda65860a15ad427": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_39722132ac22429fb0d99e5545566715",
            "placeholder": "​",
            "style": "IPY_MODEL_f7163be6070842c0abd1cc8c5af5682c",
            "value": " 500/500 [00:01&lt;00:00, 339.47ba/s]"
          }
        },
        "d1f5bcde8d754af18fdd052c98a6d81b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "d2daf033cc624958b2769a4349a04a01": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8111135a8f6f4c6b8e634fe978045a56",
            "placeholder": "​",
            "style": "IPY_MODEL_1b32d1e2681d4561840a29c55d11bc9f",
            "value": "#0: 100%"
          }
        },
        "d35f83b7433e4f1bb857789c9738bf21": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_91e3ec446c5d4030853dca426e304f9c",
            "placeholder": "​",
            "style": "IPY_MODEL_718e86634d1343559659a545f36abb42",
            "value": " 2.50k/2.50k [00:00&lt;00:00, 44.0kB/s]"
          }
        },
        "d3f574de1ff448178358f3a4ddf803c9": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "d4447eac662145188017fe281566cfea": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "d667b4d49dfe49bebbf39dba10ada96d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_ed1c5ae4218e48bc924fc615e416721f",
              "IPY_MODEL_0869911afc8549d9bd4ce0e7f97bdc0c",
              "IPY_MODEL_242c86c1e834490897262daa0ef5dfcd"
            ],
            "layout": "IPY_MODEL_389af45e82694958b597e0c25529f6c0"
          }
        },
        "d7522bc3874c4285b33ca65febd3f2d5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "d78a21a570954c9589f7780f53000985": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "d8afe82dd0454aeeb34099fd21ccf726": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "da110a0f535848838ac3018c9bd5603c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_5199e127b316467d872958d3d03fb8c5",
            "placeholder": "​",
            "style": "IPY_MODEL_1ba2f6eb67f342c4aca41c7b8624ddde",
            "value": "Downloading data: "
          }
        },
        "dc775613d7d74f4a84f0731f595eaba2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "ddb0ac228105475a83b334351aed3439": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "ddd780b848994a3eaea4257b5b3f6dee": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "de5cb07d322c4bb4ac96700939e98dae": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "df3806b3fc674e4f9f899d5dbb6a4115": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3ef01336fc77403789e6c7bf2fe7fbd4",
            "placeholder": "​",
            "style": "IPY_MODEL_6709c3d560714b06b77382f279440ef6",
            "value": "Downloading data: "
          }
        },
        "e0c61f515ead4c4b9025c7c085456131": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "e0da976d36eb42d18df29ca6ff382e34": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_f4742bc69760471090d5b0c0015e6a78",
              "IPY_MODEL_62ca2598d0ff440483940669dde3f193",
              "IPY_MODEL_2d5966ea44ec4e469a758037f90aaa3b"
            ],
            "layout": "IPY_MODEL_b4d53b1db23e46278f2a68258594c9da"
          }
        },
        "e0ea063acfe245c783f720279a8004cd": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "e2036efdcaab46719ab2b57baf01e2a2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_94d4edee3f7a406281f694241eb143ab",
            "max": 1355256,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_84ccee50a10d47f392d094a53c5223d9",
            "value": 1355256
          }
        },
        "e231320115e544239d9a800d9cd32094": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_e0c61f515ead4c4b9025c7c085456131",
            "placeholder": "​",
            "style": "IPY_MODEL_80f274ee73da4a2dbfb3d56d48636ef5",
            "value": " 7600/7600 [00:00&lt;00:00, 26960.31 examples/s]"
          }
        },
        "e24895da040e4f07ad88ded9086156b1": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "e276dfc72b4f4b42a5a93241c7dcd0b4": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_09b9bc4a5757400fb352de17b2eb1cab",
            "placeholder": "​",
            "style": "IPY_MODEL_9ef088f4f6d244738be3a42da4396635",
            "value": "Downloading: 100%"
          }
        },
        "e35b435f72d4403e9ac00852fad7eef7": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "e54ba4ac5c6949d7bcb97eb9729d6ada": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3b037f3257684895ba25778397aeee19",
            "max": 1,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_2fa2d24e07cf4c4cba4cd296ba9281b3",
            "value": 0
          }
        },
        "e54c1368445541539af7f188bc6196cb": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "e799ed767acd4c5f8825bb1bbf7f7bda": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_7431585ef62e4e26a39006d3fd8aec8c",
            "max": 1416,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_ec5c607f95b3451794e8e3f6ee2a53d5",
            "value": 1416
          }
        },
        "e84414f3f9a2410c8983e332bed42526": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "e86fce17da1142cf892cad26a42c5610": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "e921dd034a844b888523be3de972db48": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_d2daf033cc624958b2769a4349a04a01",
              "IPY_MODEL_5ca4915e26b34b1d8d125f63a9b3f8ff",
              "IPY_MODEL_8975b0dad9b34d4bb3d654e833af9915"
            ],
            "layout": "IPY_MODEL_62930768884441de957525daceb027de"
          }
        },
        "e98979e4e72441019858ec61946f934f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_674a1753257541edb0be46d7811c6e72",
            "max": 751209,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_4826faba9b6049929733e16542881e14",
            "value": 751209
          }
        },
        "ea5c3ecdef4f46509e6dc204abee8184": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "eae0b201409d4f60bb60cbdb7ad9a217": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "eb07e619043940ecb3dd33ec3f38b03c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "ec288e953b244616befab8429457c77b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_2d966f4595464c49be7f5b323cf54b1c",
            "placeholder": "​",
            "style": "IPY_MODEL_17d2e815f3ca46fa82167ccd1eb85b2a",
            "value": "Downloading: 100%"
          }
        },
        "ec57f9449406445a9e91e35c0236da0b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "ec5c607f95b3451794e8e3f6ee2a53d5": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "ed1c5ae4218e48bc924fc615e416721f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_a303620e1ed142658667b32cee4b7e70",
            "placeholder": "​",
            "style": "IPY_MODEL_0330f9b958f34c46a34e62b47568fa77",
            "value": "Downloading: 100%"
          }
        },
        "ed3abc2e209f4590a4626efa3d2d24be": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_9780793b16aa4029ae550481db873a7d",
            "placeholder": "​",
            "style": "IPY_MODEL_eae0b201409d4f60bb60cbdb7ad9a217",
            "value": "Downloading builder script: 100%"
          }
        },
        "edd8accd2cd444ac80e7d7dd9e93c605": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "ee1d498e7e20480988712721acd42b12": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_8cbd54818cbf49e7abe5a9b404e30320",
            "placeholder": "​",
            "style": "IPY_MODEL_b2990a41d73b4f058a1cf336e78b9aea",
            "value": "Downloading: 100%"
          }
        },
        "ee2cbeb2ea6749658bfb764f181c115f": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "eea3ada87a1942458ae8bac1095f0729": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "eea82cd791c141f799b256e9023eb0c2": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_645f9c5a7dc84530a409a75b0373c283",
            "placeholder": "​",
            "style": "IPY_MODEL_b1a60018182448fc9ca922efed6a4ce4",
            "value": " 232k/232k [00:00&lt;00:00, 250kB/s]"
          }
        },
        "ef5ff713eb5c41738bdd73921418308c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_bee129673ec34431932b80148b50aab0",
            "placeholder": "​",
            "style": "IPY_MODEL_9ba8816c2f51491a8596bb33fc8998d8",
            "value": " 4.06k/4.06k [00:00&lt;00:00, 130kB/s]"
          }
        },
        "f01a9c20f21547b6926e9eb44b8f43af": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_91759f59f8604393a0d72c623b1b97af",
              "IPY_MODEL_06a4618bda4949f2ace26450b1d8052e",
              "IPY_MODEL_55783e273f894123b18c1ab84fe996a1"
            ],
            "layout": "IPY_MODEL_8f96e5e63c61463181ad8daed8f104e0"
          }
        },
        "f08a4af32b364cbaa774b179c87f737a": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f101c0bc1fc34bbdbe6b386d7858b593": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_d78a21a570954c9589f7780f53000985",
            "max": 2,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_7703ade855614a7099230146866d17e6",
            "value": 2
          }
        },
        "f1e394f5978f44de9d7f539e78699582": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f20fabbf6f064b8a8be0c99065f707b4": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "f272c61ecafb44299bcc8974f6062dba": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_2f2bbcf674964943ae306aa8d1c6bb3e",
            "placeholder": "​",
            "style": "IPY_MODEL_ac6aaa3c18cd4824971c422068858a53",
            "value": " 466k/466k [00:00&lt;00:00, 312kB/s]"
          }
        },
        "f34c4b62a07949b68758d6f6ffd1b31f": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f3e74d9727564ed3ac609ebb3ec74737": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f4318ba53a2b49ce95af89537538d815": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "f4742bc69760471090d5b0c0015e6a78": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_52ce268c3a4d4c1da647dda194d2e511",
            "placeholder": "​",
            "style": "IPY_MODEL_a1cb51aa9a3246eda0676252be68909c",
            "value": "#0: 100%"
          }
        },
        "f4d7b4451ac64e9e9cba9c79a2a5ed58": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "f62aa191dfa443678339b194a59d75ff": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f7163be6070842c0abd1cc8c5af5682c": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "f79a56e0fe024b9682ec79d134201412": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_bc51d76c3a854586b7b4832771a112bf",
            "max": 2593,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_4429b3f77f2941468d72ba1b17821d9a",
            "value": 2593
          }
        },
        "f832dc2bf7e048ccb168cc96bc8fb8f2": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f8bb3d12d0144d72b7f5bed4055d2b29": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "f8eaa261c4df44cea66453f6233555f9": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "f8eaece4960948eebf0dafe087eb252e": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_87349896035b48a99c10939bc954222b",
            "placeholder": "​",
            "style": "IPY_MODEL_0c9423eb6df1485e94e4cbee5e545df4",
            "value": " 28.0/28.0 [00:00&lt;00:00, 918B/s]"
          }
        },
        "f9ac22e242984baeaae8ddae3748003c": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "fa07548194e244b8b655381e073abb9d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "fa11c85885b34c0c8e105c3cec6fbb83": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "fae2e3a4035d403dad3baef4802a9258": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_bd052d320db8492c9b3aa1cf29e4a9cf",
              "IPY_MODEL_55ed66dd456e4f19ae98ce739da552b8",
              "IPY_MODEL_a4befd44d0b5402e8bcb61f64162ff43"
            ],
            "layout": "IPY_MODEL_f08a4af32b364cbaa774b179c87f737a"
          }
        },
        "fb68c9499ecb4403939127cd750718eb": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_75efbc49c6e84d9f8146c04de0f4daa8",
            "placeholder": "​",
            "style": "IPY_MODEL_4f35888d5ef74153b2c1bf739cce1f9c",
            "value": " 2/2 [00:00&lt;00:00, 48.02it/s]"
          }
        },
        "fc44ba4fc74f472a95cbeab1ea179b89": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "fc631832218144c4b29f776ebba56193": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HBoxModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_0f938c22289d45d9bdd7929ed1a97626",
              "IPY_MODEL_b9474845c2d24c6f91420bb23ca0e149",
              "IPY_MODEL_2066c6d9fe2a4d6f876e81fbbf64e653"
            ],
            "layout": "IPY_MODEL_868d87d61432416fb23969b8405cc628"
          }
        },
        "fd2c6d3ac0754db3acad1e88ade2c86d": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "FloatProgressModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_89c56c8131c74168ad32930ab11500f1",
            "max": 380,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_96906f2b2dcf4a34bcb6418d44751575",
            "value": 380
          }
        },
        "fd921171897a4ffc99562d675733b308": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "fda6478d993342a798d281d6da9d8cd0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "ProgressStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "fe48ae258e904d6aaac09c728868ce41": {
          "model_module": "@jupyter-widgets/base",
          "model_module_version": "1.2.0",
          "model_name": "LayoutModel",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "fee18278d97f4191a086db8dc185409b": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "DescriptionStyleModel",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "feed7ab81f894f04b4e485f54ad25ff0": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_2491724422214716972d8198d78dba2e",
            "placeholder": "​",
            "style": "IPY_MODEL_01ae48366913491c910b0587be32596c",
            "value": " 1.42k/1.42k [00:00&lt;00:00, 44.1kB/s]"
          }
        },
        "ff32978b71364a37a6724a5ee508bed3": {
          "model_module": "@jupyter-widgets/controls",
          "model_module_version": "1.5.0",
          "model_name": "HTMLModel",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_6835c929cd844a7f882765f68bd8a7e2",
            "placeholder": "​",
            "style": "IPY_MODEL_1de84e67a3b34cddb1c73beae7888833",
            "value": "Downloading: 100%"
          }
        }
      }
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}